# 학과 데이터 입력
#*입력되는 엑셀의 기본 조건

## 1. 제목 행은 총 4행 > 템플릿 파일들 기준으로 맞추자!
- 1행 : 2023년_ [4-마. 재적 학생 현황_] 같은 설명
- 2~3행 : 빈행 (체크 필요)
- 4행 ~ 7행 : 제목행 (셀들은 병합되어있으며 변경됨)

## 2. 삭제되는 공통열은 university 테이블 참조
- '설립구분'
- '학교종류'
- '지역'
- '상태'

## 3. 엑셀의 컬럼명과 일치하는 컬럼명으로 자동 매칭되도록...

In [75]:
import os, sys
# 판다스
#!pip install pandas
#%pip install pandas

# Pandas
#%pip install pandas
import pandas as pd

#source / scema.py
import source.schema as sch
# print(sch.table_dic['contract_department'])
# sys.exit(0)


#################################################
# 디버그 모드 설정
#################################################
def _dbg_(a='', b='', c='', d='', e='', f=''):
    print('[dbg]', '(', sys._getframe(1).f_code.co_name, ') ', a, b, c, d, e, f)
    
def _info_(name, var):
    if isinstance(var, str): 
        print('[info]', '(', sys._getframe(1).f_code.co_name, ') ', '"', name, '" | ', type(var), ' | ', var)
    elif isinstance(var, int): 
        print('[info]', '(', sys._getframe(1).f_code.co_name, ') ', '"', name, '" | ', type(var), ' | ', var)
    else:
        print('[info]', '(', sys._getframe(1).f_code.co_name, ') ', '"', name, '" | ', type(var), ' | ', len(var), ' | ', var)
    #[info] ( find_key3_k )  <class 'str'> 8 key_list

In [76]:
#################################################
# DB 접속 및 처리 설정
#################################################

#%pip install pymysql
import pymysql

def db_setting(ip, id, pwd, dbname):
    conn = pymysql.connect(host=ip, user=id, password=pwd, db=dbname)
    curs = conn.cursor(pymysql.cursors.DictCursor)

    if conn == None:
        print('DB 연결 오류')
    elif curs == None:
        print('curs 연결 오류')
        
    return conn, curs

#################################################
# 테이블의 존재 여부를 확인
#################################################
def chk_table(curs, dbname, table):
    #_dbg_(table)
    sql = "select 1 from INFORMATION_SCHEMA.TABLES where TABLE_SCHEMA = '{}' AND TABLE_NAME = '{}'".format(dbname, table)
    #print('chk_table : ', sql)
    ret = curs.execute(sql)
    
    if ret != 1:
        _dbg_('{}.{} 테이블이 없으므로 생성해야함'.format(dbname, table))
    else:
        _dbg_('{}.{} 테이블이 존재함'.format(dbname, table))

    return ret


#################################################
# DB 테이블의 프라이머리 키값만 추출
#################################################
def get_db_primary_keys(curs, dbname, table_name):
    # try:
    sql = f"SHOW KEYS FROM {table_name} WHERE Key_name = 'PRIMARY'"
    curs.execute(sql)
    primary_keys = curs.fetchall()
    pk = []
    
    # 키 필드 정보 출력
    if primary_keys:
        _dbg_(f"테이블 '{table_name}'의 기본 키 필드:")
        for key in primary_keys:
            _dbg_("컬럼명:", key['Column_name'])
            pk.append(key['Column_name'])
    else:
        _dbg_(f"테이블 '{table_name}'에는 기본 키가 없습니다.")
        
    
    return pk
    # except mysql.connector.Error as err:
    #     print("데이터베이스 오류:", err)

    # finally:
    #     # 연결 종료
    #     if conn.is_connected():
    #         cursor.close()
    #         conn.close()
    
#################################################
# DB 테이블의 외래 키값만 추출
#################################################
def get_db_foregn_keys(curs, dbname, table_name):
    # try:
    sql = f"""
    SELECT
        COLUMN_NAME,
        REFERENCED_TABLE_NAME,
        REFERENCED_COLUMN_NAME
    FROM
        information_schema.KEY_COLUMN_USAGE
    WHERE
        TABLE_NAME = '{table_name}'
        AND TABLE_SCHEMA = '{dbname}'
        AND REFERENCED_COLUMN_NAME IS NOT NULL;
    """
    
    curs.execute(sql)
    foreign_keys  = curs.fetchall()
    
    # 외래 키 정보 출력
    if foreign_keys:
        _dbg_(f"테이블 '{table_name}'의 외래 키 정보:")
        for fk in foreign_keys:
            _dbg_("컬럼명:", fk['COLUMN_NAME'])
            _dbg_("참조 테이블:", fk['REFERENCED_TABLE_NAME'])
            _dbg_("참조 컬럼:", fk['REFERENCED_COLUMN_NAME'])
            _dbg_("-" * 30)
    else:
        print(f"테이블 '{table_name}'에는 외래 키가 없습니다.")
        
    # except mysql.connector.Error as err:
    #     print("데이터베이스 오류:", err)

    # finally:
    #     # 연결 종료
    #     if conn.is_connected():
    #         cursor.close()
    #         conn.close()
        
    
#################################################
# DB 테이블의 컬럼명과 데이터타입만 추출
#################################################
def get_db_column_name(curs, dbname, table):
    _dbg_(table)
    #cnt_columns = len(df_excel.columns) #엑셀 df읠 필드명

    sql_col = 'SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = "{}"'.format(table)

    curs.execute(sql_col)
    #conn.commit()
    col_info = curs.fetchall() #컬럼명과 데이터타입 리스트 안에 딕셔너리로...
    
    _dbg_(table, '테이블의 컬럼 정보 : ', type(col_info), len(col_info), col_info[:8])

    idx_mode = 'off' #기준연도+학교코드 모드와 idx 모드로 나뉨
    col_name_list = list()
    col_name_str = ''
    col_cnt = len(col_info)
    
    for n in range(col_cnt):
        col = col_info[n]['COLUMN_NAME']
        #_dbg_('[{}]'.format(col))
        if col == 'idx':
            idx_mode = 'on'
        else:
            #col_name_list.append(col_name[n]['COLUMN_NAME'])
            col_name_list.append(col)
            #_dbg_('appended : [{}]'.format(col))

    if idx_mode == 'on': #idx하나를 빼야 함
        col_cnt = col_cnt - 1

    for n in range(col_cnt):
        if n == 0:
            col_name_str = '`' + str(col_name_list[n]) + '`, '
        elif n >= col_cnt-1:
            col_name_str = col_name_str + '`' + str(col_name_list[n]) + '`'
        else:
            col_name_str = col_name_str + '`' + str(col_name_list[n]) + '`, '


    #print(col_name_str)
    # print(type(col_name), len(col_name), col_name)
    # print(type(col_name[0]), len(col_name[0]), col_name[0])
    # print(type(col_name[0]['COLUMN_NAME']), col_name[0]['COLUMN_NAME'])

    return col_info, col_name_str, col_name_list


#################################################
# 데이터 프레임의 의 컬럼명만 추출
#################################################
# def get_df_column_name(curs, dbname, table):



#################################################
# 테이블 필드명의 데이터 타입을 얻어옴
#################################################
def get_data_type(col_info, col_name):
    data_type = ''
    for info in col_info:
        #print(info['COLUMN_NAME'], info['DATA_TYPE'])
        if info['COLUMN_NAME'] == col_name:
            data_type = info['DATA_TYPE']
        
    return data_type


# {COLUMN_NAME': 'idx', 'DATA_TYPE': 'int'}
# {'COLUMN_NAME': 'year', 'DATA_TYPE': 'int'}
# {'COLUMN_NAME': 'univ_cd', 'DATA_TYPE': 'varchar'}


#################################################
# 문서명에 해당하는 테이블 명을 확인, 띄어쓰기 모두 포함된 상태
#################################################
def get_table_name(doc_title):
    _dbg_('[{}]'.format(doc_title))
    _dbg_(sch.table_dic.get(doc_title))
    
    if sch.table_dic.get(doc_title) == None:
        _dbg_('[종료] [{}]를 저장할 테이블 구조가 없으므로 [table_dic]에 구조부터 생성해야함'.format(doc_title))
        sys.exit(0)
    else:
        table_name = sch.table_dic[doc_title]
        _dbg_('[{}]를 저장할 [{}] 테이블 존재함'.format(doc_title, table_name))
    
    return table_name


# import mysql.connector
#################################################
# 데이터베이스의 모든 테이블 명 읽기
#################################################
def get_db_table_name(dbname, connection):
    
    table_name = []
    try:
        cursor = connection.cursor()
        # 테이블 이름 가져오기
        cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{}'".format(dbname))
        tables = cursor.fetchall() #tuple

        # 테이블 이름 출력
        #_dbg_("테이블 목록:", type(tables), tables)
        for table in tables:
            #print(table[0])
            table_name.append(table[0])
            
    finally:
        cursor.close()
        connection.close()
        
    return table_name #list


#################################################
# 데이터베이스의 모든 테이블 명 읽기
#################################################
def get_db_table_comment(dbname, connection):
    try:
        cursor = connection.cursor()
        
        # 테이블 주석 가져오기
        cursor.execute("""
            SELECT TABLE_NAME, TABLE_COMMENT
            FROM INFORMATION_SCHEMA.TABLES
            WHERE TABLE_SCHEMA = '{}';
        """.format(dbname))
        table_comments = cursor.fetchall()
        
        #tbl_dict = {"name": '', "comment": '', "year": 0, "year_cnt" : 0}
        tbl_dict = {}
        for table_name, comment in table_comments:
            #tbl_dict["name"] = table_name
            #tbl_dict["comment"] = comment
            #tbl_dict = dict(name=table_name, comment=comment, year=0, year_cnt=0)
            
            #tbl_dict[table_name] = {"comment" : comment}
            tbl_dict[table_name] = comment
            
            #print(f"{table_name} : {comment}")
            #print(type(table_name), type(comment))

        # 특정 테이블의 컬럼 주석 가져오기
        # cursor.execute("""
        #     SELECT COLUMN_NAME, COLUMN_COMMENT
        #     FROM INFORMATION_SCHEMA.COLUMNS
        #     WHERE TABLE_SCHEMA = '{}' AND TABLE_NAME = '{}';
        # """.format(dbname, table_name))
        # column_comments = cursor.fetchall()
        # print("\n컬럼 주석:")
        # for column_name, comment in column_comments:
        #     print(f"{column_name}: {comment}")
        
    finally:
        cursor.close()
        #connection.close()

    return tbl_dict

#################################################
# VIEW테이블 여부 확인
#################################################
def get_db_view_chk(dbname, connection, tbl_name):
    try:
        cursor = connection.cursor()
        
        # 특정필드의 고유한 값 갯수 카운트
        cursor.execute("""
            SELECT TABLE_TYPE
            FROM INFORMATION_SCHEMA.TABLES
            WHERE TABLE_SCHEMA = "{}"
            AND TABLE_NAME = "{}";
        """.format(dbname, tbl_name))
        
        ret = False
        result = cursor.fetchone()
        if result and result[0] == 'VIEW' or result[0] == 'view':
            #_dbg_('{} 테이블은 VIEW 테이블임'.format(tbl_name))
            ret = True
        else:
            #_dbg_('{} 테이블은 일반 테이블임'.format(tbl_name))
            ret = False
                
    finally:
        cursor.close()
        #connection.close()
        
    return ret


#################################################
# 특정 컬럽이 테이블에 존재하는지 확인
#################################################
def get_db_column_chk(dbname, connection, tbl_name, col_name):
    try:
        cursor = connection.cursor()
        
        ##############################################
        #year 컬럼이 없는 건 패스해야함
        #전체 테이블 수와 리스트 출력
        #각 년도별 카운트 해서 이상유무 검증
        
        # 특정필드의 고유한 값 갯수 카운트
        cursor.execute("""
            SELECT COUNT(*)
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_SCHEMA = "{}"
            AND TABLE_NAME = "{}"
            AND COLUMN_NAME = "{}";
        """.format(dbname, tbl_name, col_name))
        
        ret = False
        result = cursor.fetchone()
        if result[0] > 0:
            #_dbg_('{} 테이블에 {} 컬럼이 존재'.format(tbl_name, col_name))
            ret = True
        else:
            #_dbg_('{} 테이블에 {} 컬럼이 존재하지 않음'.format(tbl_name, col_name))
            ret = False
                
    finally:
        cursor.close()
        #connection.close()
        
    return ret

#################################################
# GROUP BY 절을 사용하여 특정 컬럼 값을 기준으로 그룹별 고유한 레코드 수를 확인
#################################################
def get_db_year_cnt(dbname, connection, tbl_name, col_name):
    try:
        cursor = connection.cursor()
        
        # 특정필드의 고유한 값 갯수 카운트, 2021, 2022, 2023 --> 3
        # cursor.execute("""
        #     SELECT COUNT(DISTINCT {}) AS unique_count
        #     FROM {};
        # """.format(col_name, tbl_name))
        # col_uniq_cnt = cursor.fetchone()[0]
        # print(f"레코드 수: {col_uniq_cnt}")
        
        # 특정필드의 고유한 값 갯수 카운트
        cursor.execute("""
            SELECT {}, COUNT(*) AS record_count
            FROM {}
            GROUP BY {};
        """.format(col_name, tbl_name, col_name))
        col_cnt_tuple = cursor.fetchall()
        #print(f"레코드 수: {col_cnt_tuple}", type(col_cnt_tuple))
        
    finally:
        cursor.close()
        #connection.close()

    return col_cnt_tuple


#################################################
# 테이블 생성
#################################################
def create_table(curs, dbname, table):
    _dbg_(table)
    ret_val = ''
    ret_mode = '' #idx 가 있냐 없냐...

    sql = sch.table_dic[table]
    chk_tbl = chk_table(curs, dbname, table)
    if chk_tbl != 1: #테이블명에 해당하는 테이블이 없음
        _dbg_('create_table : ', sql)
        rs = curs.execute(sql)
        
        if rs !=0 :
            _dbg_('[오류_종료]create_table 생성오류 [{}]: {}'.format(rs, table))
            os.exit(0)
        else:
            _dbg_('{}.{} 신규 테이블 생성 성공'.format(dbname, table))
            ret_val = 'create'
    elif chk_tbl == 1:
        _dbg_('{}.{} 테이블이 존재함'.format(dbname, table))
        ret_val = 'exist'
    else:
        _dbg_('[오류_종료] 알수없는 생성오류 [{}]: {}'.format(rs, table))
        
    #_dbg_(type(sql), sql)
    
    if sql.find('`idx` int auto_increment') > 0:
        _dbg_('idx mode')
        ret_mode = 'idx_on'
    else:
        _dbg_('기준년도+학교코드 mode')
        ret_mode = 'idx_off'
    #    테이블에 idx가 있는 경우는 다르게 처리
    #    테이블에 idx가 있다면 인서트_업데이트를 다르게..
        
        
    return ret_val, ret_mode


#################################################
# university테이블에서 특정 또는 가장 큰 코드값 읽기
#################################################
def get_univ_maxcode(curs):
    _dbg_()
    
    sql = 'SELECT {} FROM {}'.format('univ_cd', 'university') #university 테이블에서 가장큰 univ_code값을 가져오기 위함
    curs.execute(sql)
    code = curs.fetchall() #university에 등록된 대학교 목록
    
    max_code_val = 0
    
    for cd in code:
        #print('학교코드', type(cd), cd, cd['univ_cd'])
        
        code_str = cd['univ_cd']
        if code_str.find('-') == -1: #특수 기호 없다면
            max_code_val = max(max_code_val, int(code_str))
            
    #기존 테이블에 최대값은 10000x으로 되기 때문에 문자열로해도 100000x
    max_code_val += 1
    return str(max_code_val)


#################################################
# 대학코드 -> 대학명
#################################################
def get_univ_name(curs, univ_code):
    _dbg_(univ_code)
    sql = 'SELECT {} FROM {} WHERE univ_cd = {}'.format('univ_name', 'university', univ_code) #university 테이블 기준으로 매핑하기 때문에 고정 값
    curs.execute(sql)

    univ_name = curs.fetchall() #university에 등록된 대학교 목록

    return univ_name
        
        
#################################################
# 대학명 -> 대학코드
#################################################
def get_univ_name(curs, univ_name):
    _dbg_(univ_name)
    sql = 'SELECT {} FROM {} WHERE univ_name_arlimi = {}'.format('univ_cd', 'university', univ_name) #university 테이블 기준으로 매핑하기 때문에 고정 값
    curs.execute(sql)

    univ_code = curs.fetchall() #university에 등록된 대학교 목록

    return univ_code
    

In [77]:
##################################################################################################
##################################################################################################
# 단독 작업
# DB에 입력된 데이터 검증 : 테이블명, 커맨트, 년도, 년도별 입력된 갯수 확인
##################################################################################################
##################################################################################################
def get_db_data_check(dbname, conn):
    #dbname = 'swu_external'

    #conn, curs = db_setting('133.186.247.101', 'root', 'coss!@#$', dbname = 'cosshubtest') #coss test
    #conn, curs = db_setting('13.124.114.176', 'root', 's!W@u#12', dbname = dbname)

    table_cnt = 0

    #전체 테이블 명과 커멘트 읽기
    tbl_dict = get_db_table_comment(dbname, conn) #"name": '', "comment": '', "year": 0, "year_cnt" : 0

    #print(tbl_dict.keys()) #dict_keys(['name', 'comment', 'year', 'year_cnt'])
    #print(tbl_dict.values()) #dict_values(['financial_budget', '자금예산서(예산)', 0, 0])
    
    #작업 대상 테이블이 아님
    no_work_list = ['swu_industry_overhead_amount', 'swu_fund', 'group_univ', 'del_financial_budget', 'del_income_statement', 'del_financial_settlement', 'del_balance_sheet']

    #테이블명과 커멘트
    for key, value in tbl_dict.items():
        #print(tbl_dict.get("financial_budget"))
        
        if key not in no_work_list:
            # 'year'필드가 있어야하고 view테이블은 아니어야함
            if get_db_column_chk(dbname, conn, key, 'year') == True and get_db_view_chk(dbname, conn, key) == False:
                _dbg_(f"{key} | {value}")
                #특정 테이블에서 year에 해당하는 컬럼의 갯수 카운트
                col_cnt_tuple = get_db_year_cnt(dbname, conn, key, 'year')
                
                for index, value in enumerate(col_cnt_tuple):
                    #print(f"인덱스 {index}: {value}")
                    _dbg_(f"{value}")
                    
                table_cnt += 1
                #print(f"연도별 데이터 수: {col_cnt_tuple}") #, type(col_cnt_tuple))
        
    return table_cnt
    # 영문테이블명 | 한글테이블명(커멘트) | 년도 | 년도별 입력 갯수


In [78]:
#sys.exit(1)

In [79]:
#################################################
# 물리적 파일명 검증, 각 디렉토리에서 일치하는 파일명 기준으로 일괄 작업 수행
#################################################

#디렉토리의 전체 파일명 읽기
def read_filename(dir): 
    file_list = os.listdir(dir)
    return file_list

#전체 경로에서 파이명만 추출
def extract_filename(full_name): 
    file_name = os.path.basename(full_name)
    return file_name


#################################################
# 유틸 : 문자열내 공백 제거, 문자열 교체
#################################################
def refine_univ_name(str):
    # A단계에서 처리가 안되는경우 2차 예외처리
    # 강릉원주대학교 vs 국립강릉원주대학교 와 같은 식으로 프리픽스와 포스트가 다른 경우가 있다.
    # 하나의 코드명으로 처리해야하기 때문에 유사한 대학명을 동일하게 매핑해야함            
    # 1. 국립 : 국립강릉원주대학교, 경상국립대학교와 같은 패턴, # 2. 숫자로 된 캠퍼스는 숫자만 일치해도 동일하게 처리

    if str is not None:
        str = str.lstrip()
        str = str.rstrip()
        str = str.replace('국립', '') #국립제거
        str = str.replace(' ', '') #띄어쓰기제거
    return str


In [ ]:
#################################################
# df관련 처리 작업
#################################################


#################################################
# df의 헤더 및 제목정보 출력
#################################################
def df_info(str, df):
    df_header = list(df.columns) #컬럼정보
    print('[dbg]', str, '헤더 : ', df_header[:8], '...')
    print('[dbg]', str, 'shape : ', df.shape[0], '행', df.shape[1], '열')
    
#################################################
# df의 행, 열 정보 리턴
#################################################
def get_df_shape(df):
    return df.shape[0], df.shape[1]
    

#################################################
# 숫자 셀중 , - " 등이 들어간 경우가 있기 때문에 숫자값을 정규화
#################################################
def norm_value(var, remove_char):
    result = var
    
    if result == '-':
        result = 0
        return 0
    
    if isinstance(result, str): #문자열만 작업 수행
        for ch in remove_char:
            result = result.replace(ch, '')
        
    return result


#################################################
# 템플릿데이터와 입력된 데이터 검증
# return doc_title : 해당 문서명을 기준으로 테이블 생성여부 확인
#################################################
def compare_schema(template, excel, sheet_name): 
    _dbg_('템플릿 파일 : ', template, sheet_name)
    _dbg_('엑셀 파일 : ', excel, sheet_name)
    # excel file load
    #%pip install openpyxl
    df_template = pd.read_excel(template, sheet_name = sheet_name) #템플릿과 일치여부를 확인하자
    df_excel = pd.read_excel(excel, sheet_name = sheet_name)

    # 데이터프레임의 nan 값을 None으로 변경
    #df.dropna(inplace=True)
    #df = df.where((pd.notnull(df)), 'None')
    df_template = df_template.fillna('None')
    df_excel = df_excel.fillna('None')
    
    
    ###################################
    # 첫줄 컬럼 헤더 비교
    ###################################
    df_template_header = list(df_template.columns) #템플릿 헤더
    df_excel_header = list(df_excel.columns) #df 헤더
    
    #_dbg_(df_excel_header[0]) #['2023년_ [4-다. 신입생 충원 현황_]        doc_title :  7 23 2023년_ [4-다. 신입생 충원 현황_]             doc_title :  11 22 2023년_ [4-다. 신입생 충원 현황_] +1 -1
    
    #법인 어쩌고 검증
    # [8-마-1. 자금예산서(예산)_] -> [8-마-1. 법인_자금예산서(예산)_]
    # [8-마-2. 자금계산서(결산)_] -> [8-마-2. 법인_자금계산서(결산)_]
    # [8-마-3. 운영(손익) 계산서_] -> [8-마-3. 법인_운영(손익) 계산서_]
    # [8-마-4. 대차대조표_] -> [8-마-4. 법인_대차대조표_]
    
    if '[8-마-1. 자금예산서(예산)_]' in df_excel_header[0]:
        df_excel_header[0] = '[8-마-1. 법인_자금예산서(예산)_]'
        _dbg_('!!! 제목행의 [8-마-1. 자금예산서(예산)_] -> [8-마-1. 법인_자금예산서(예산)_] 로 변경')
    elif '[8-마-2. 자금계산서(결산)_]' in df_excel_header[0]:
        df_excel_header[0] = '[8-마-2. 법인_자금계산서(결산)_]'
        _dbg_('!!! 제목행의 [8-마-2. 자금계산서(결산)_] -> [8-마-2. 법인_자금계산서(결산)_] 로 변경')
    elif '[8-마-3. 운영(손익) 계산서_]' in df_excel_header[0]:
        df_excel_header[0] = '[8-마-3. 법인_운영(손익) 계산서_]'
        _dbg_('!!! 제목행의 [8-마-3. 운영(손익) 계산서_] -> [8-마-3. 법인_운영(손익) 계산서_] 로 변경')
    elif '[8-마-4. 대차대조표_]' in df_excel_header[0]:
        df_excel_header[0] = '[8-마-4. 법인_대차대조표_]'
        _dbg_('!!! 제목행의 [8-마-4. 법인_대차대조표_] -> [8-마-4. 법인_대차대조표_] 로 변경')
            
    
    
    #doc_title = df_excel_header[0] #엑셀기준 1행의 문서명
    doc_title = df_template_header[0] #템플릿기준 1행의 문서명 > 스키마의 이름과 일치해야하므로...
    pos_1 = doc_title.find('. ')
    pos_2 = doc_title.find('_]')
    
    if pos_1 <= 0 or pos_2 <= 0:
        _dbg_('[종료] 엑셀 문서 제목 확인 필요 ', df_excel_header[0])
        sys.exit(0)
        
    doc_title = doc_title[pos_1+2:pos_2]
    #_dbg_('doc_title : ', pos_1, pos_2, df_excel_header[0], '[' + doc_title + ']') #['2023년_ [4-다. 신입생 충원 현황_]             doc_title :  11 22 2023년_ [4-다. 신입생 충원 현황_] [ 신입생 충원 현황]
    _dbg_('문서 제목 및 스키마참고하는 테이블명 : ', df_excel_header[0], '[' + doc_title + ']') #['2023년_ [4-다. 신입생 충원 현황_]             doc_title :  11 22 2023년_ [4-다. 신입생 충원 현황_] [ 신입생 충원 현황]
    
    df_template_header[0].find('[', 0)
    df_excel_header[0].find('[', 0)

    df_template_header[0] = '' #첫행 첫열은 0000년_ [2-가-1. 전공과목 성적 분포_]
    df_excel_header[0] = ''
    
    #_dbg_('df_tmplt_header', df_template_header)
    #_dbg_('df_excel_header', df_excel_header)
    
     
    if df_template_header != df_excel_header:
        _dbg_('[종료] : 첫줄 데이터 구조가 다름(1행)')               ############ ryu, 수작업 추가 : 2022년_ [12-라-2. 외국대학과 학점 교류 현황_] 과 2023년_ [12-라-2. 외국대학과 학점 교류 현황_]의 헤더가 다름
        _dbg_('df_tmplt_header : '. df_template_header[0])
        _dbg_('df_excel_header : '. df_excel_header[0])
        sys.exit(0)

    #_dbg_('df_template', type(df_template), df_template)
    #_dbg_('df_template.shape[0]', type(df_template.shape[0]), df_template.shape[0])


    ###################################
    # 2행 이하의 제목행 비교
    ###################################
    df_template_shape0 = df_template.shape[0]
    
    for n in range(df_template_shape0):
        _dbg_('n / df_template_shape0', n, df_template_shape0)
        template_rows_list = list(df_template.loc[n]) #컬럼 시리즈를 리스트로 변경
        excel_rows_list = list(df_excel.loc[n]) #컬럼 시리즈를 리스트로 변경
        
        _dbg_(n, 'tmplt_rows_list', type(template_rows_list), template_rows_list)
        _dbg_(n, 'excel_rows_list', type(excel_rows_list), excel_rows_list)
        
        
        #셀중 \n에 포함된 경우 제거하는 작업
        template_rows_list_mod = [word for word in template_rows_list if "\n" not in word]
        excel_rows_list_mod = [word for word in excel_rows_list if "\n" not in word]
        
        # _dbg_(n, 'tmplt_rows_list_mod', type(template_rows_list_mod), template_rows_list_mod)
        # _dbg_(n, 'excel_rows_list_mod', type(excel_rows_list_mod), excel_rows_list_mod)
        
        #excel_rows_list에서 '대학(원)명'을 '학교명'으로 변경
        if '대학(원)명' in excel_rows_list_mod:
            index = excel_rows_list_mod.index('대학(원)명')
            excel_rows_list_mod[index] = '학교명'
            _dbg_('!!! 엑셀의 대학(원)명을 학교명으로 변경처리')


        if template_rows_list_mod != excel_rows_list_mod:
            _dbg_('[종료] : {}번째 줄 데이터 구조가 다름(1행)'.format(n))
            _dbg_('tmplt_rows_list : ', template_rows_list_mod)
            _dbg_('excel_rows_list : ', excel_rows_list_mod)            
            sys.exit(0)
        # else:
        #     _dbg_(n, 'tmplt_rows_list_mod2', type(template_rows_list_mod), template_rows_list_mod)
        #     _dbg_(n, 'excel_rows_list_mod2', type(excel_rows_list_mod), excel_rows_list_mod)
            

        _dbg_('일단 통과')
    
    _dbg_('성공 : 두개의 작업문서 구조 일치함')
    #_dbg_('문서 헤더 - df_tmplt_header : ', df_template_header)
    #_dbg_('문서 shape : ', df_template.shape[0], '행', df_template.shape[1], '열')
    #df_info('df_template', df_template)
    return df_excel, doc_title, df_template_shape0


#################################################
# df_excel에서 2~3번째 빈행 삭제 후 헤더재설정
#################################################
def remove_empty_row1(df_excel, df_template_shape0=0):
    _dbg_('df_tmplt_shape0 : ', df_template_shape0)

    #_dbg_('df_excel.info() : ', type(df_excel.info()), df_excel.info())
    
    #현재 헤더 정보 확인
    header_info = df_excel.columns.tolist()
    #_dbg_('df_excel의 헤더 : ', header_info)
    df_info('원본 df_excel 헤더', df_excel)
    
    
    excel_rows_list = []
    
    new_header = []
    new_header_cnt = 0
    
    ## 학교 > 학교명 등 특정잘못된 제목부터 변경
    # 한번 추가한 헤더 컬럼을 또 추가하지 않기 위해 플랙 추가... 일부 학교명과 학교가 중보되어 들어간 문서도 있다. 2023년 _대학_14-마-2-(1). 기숙사 운영 결과(국·공립_민자)_학교별자료.xlsx
    flag_year = 0
    flag_utype = 0
    flag_estblish = 0
    flag_local = 0
    flag_status = 0
    flag_uname = 0
    flag_corp = 0
    
    for n in range(df_template_shape0): #템플릿 엑셀기준 행 전체정보 확인
        excel_rows_list = list(df_excel.loc[n]) #컬럼 시리즈를 리스트로 변경
        _dbg_(n, '행 데이터 : ', excel_rows_list[:8], "...")
        for m in range(len(excel_rows_list)): #각 열 순회
            cell = excel_rows_list[m]
            cell = cell.replace("\n", "")
            #_dbg_('cell 값 : ', cell)
            if cell == '기준연도' or  cell == '연도' and flag_year == 0:
                new_header.append('기준연도')
                new_header_cnt += 1
                flag_year = 1
            elif cell == '학교종류' and flag_utype == 0:
                new_header.append('학교종류')
                new_header_cnt += 1
                flag_utype = 1
            elif cell == '설립구분' and flag_estblish == 0:
                new_header.append('설립구분')
                new_header_cnt += 1
                flag_estblish = 1
            elif cell == '지역' and flag_local == 0:
                new_header.append('지역')
                new_header_cnt += 1
                flag_local = 1
            elif cell == '상태' or cell == '상태정보' and flag_status == 0:
                new_header.append('상태')
                new_header_cnt += 1
                flag_status = 1
            elif cell == '학교명' or cell == '학교' or cell == '대학(원)명' and flag_uname == 0: #대학(원)명 : 6-가-2. 전체 신규 전임교원 현황_학교별자료
                new_header.append('학교명')
                new_header_cnt += 1
                flag_uname = 1
            elif cell == '법인명' and flag_uname == 0:
                new_header.append('법인명')
                new_header_cnt += 1
                flag_corp = 1
                
    #나머지 new_header 정의
    for m in range(len(excel_rows_list)-new_header_cnt): #각 열 순회
        hdr = 'new-hdr-{}'.format(m)
        new_header.append(hdr)

    #새로운 헤더 지정
    if len(new_header) <= 0:
        _dbg_('오류 : 새로운 헤더를 생성하지 못함 : ', new_header)
        sys.exit(0)
        
    df_excel.columns = new_header
    _dbg_('새로운 헤더를 지정함 : ', new_header[:8])

    #기준연도 열 정보 읽기
    find_year = ''
    year_col = df_excel['기준연도'].to_list()
    #_dbg_('year_col', type(year_col), year_col)

    for str in year_col:
        if str != '기준연도' and str != 'None' and str != 'none' and str != '':
            find_year = str
            

    if find_year == '':
        _dbg_('오류 : 연도를 찾을 수 없음 ', find_year)
        sys.exit(0)
    else:
        _dbg_('성공 : 발견한 기준년도 = ', find_year)


    #나머지 제목행들 삭제
    #일부 2023년 _대학_4-가. 입학전형 유형별 선발 결과_학교별자료.xlsx 처럼 엑셀제목행은 6행으로 병합된었지만 실제는 5행인 경우가 있음, 
    #따라서 아래 몇행정도를 더 확인 후 None들로만 채워진 행 정보를 확인해서 삭제 해야함
    search_more_row = 5
    for n in range(df_template_shape0+search_more_row):
        excel_rows_list = list(df_excel.loc[n]) #컬럼 시리즈를 리스트로 변경

        if n < df_template_shape0:
            _dbg_('{}행 - 삭제 : {}, "..."'.format(n, excel_rows_list[:18]))
            df_excel = df_excel.drop(n) #빈행 삭제
        else: #삭제 후 엑셀제목행 5개정도 아래의 None값들의 수를 확인해보자    
            #_dbg_('excel_rows_list : ', excel_rows_list)
            none_cnt = excel_rows_list.count('None')
            row_len = len(excel_rows_list)
            if none_cnt == row_len: # None행임
                _dbg_('{}행 - 삭제 : {}, "..."'.format(n, excel_rows_list[:18]))
                df_excel = df_excel.drop(n) #빈행 삭제
                
    

    df_excel = reset_index(df_excel)

    ## 삭제 후 정리된 헤터 정보 출력
    df_info('( remove_empty_row1 ) 정리된 df_excel ', df_excel)
    

    # ## 삭제 후 정리된 행정보 출력
    # for n in range(df_template_shape0): #템플릿 엑셀기준 행정보 확인
    #     #_dbg_(df_template_shape0, n)
    #     excel_rows_list = list(df_excel.loc[n]) #컬럼 시리즈를 리스트로 변경
    #     _dbg_(n, excel_rows_list)

# header_mapping = {
#     'old_header1': 'new_header1',
#     'old_header2': 'new_header2',
#     # Add more mappings as needed
# }

# # Rename the headers using the dictionary
# df_excel.rename(columns=header_mapping, inplace=True)

    return df_excel, find_year

#################################################
# df 헤더명이 다른 경우가 있음 학교명 vs 학교...
#################################################
#def refine_header(df_excel):
    # _dbg_()

    # df_excel_header = list(df_excel.columns) #df 헤더

    # #엑셀의 [1,1]은 2023년_ [4-자. 신입생의 출신 고등학교 유형별 현황_]
    # df_excel_header[0].find('[', 0)
    # df_excel_header[0] = '기준연도'
    
    # for n in range(len(df_excel_header)):
    #     if df_excel_header[n] == '학교':
    #         df_excel_header[n] = '학교명'
            
    # _dbg_('df_excel_header', df_excel_header)
    
    # #타이틀 컬럼명 재지정
    # df_excel.columns = list(df_excel_header)
    
    #return df_excel


#################################################
# df_excel에서 2~3번째 빈행 삭제 후 헤더재설정
#################################################
def remove_empty_row2(df_excel):

    index_values = list(df_excel.index.values) #전체 행 인덱스 정보
    
    for idx in index_values:
        cell_year = df_excel.loc[idx, '기준연도']
        cell_univ = df_excel.loc[idx, '학교명']
        #print('remove_empty_row2 (cell_univ) : ', cell_univ)
        cell_type = df_excel.loc[idx, '학교종류']
        
        if cell_year == '기준연도' and cell_univ == '학교명' and cell_type == '학교종류':
            _dbg_('{}행 삭제'.format(idx), cell_year, cell_univ, cell_type)
            df_excel = df_excel.drop(idx) #제목행 복사로 반복행 제거
        elif cell_year == '기준연도' and cell_univ == '학교명' or cell_univ == '학교':
            _dbg_('{}행 삭제'.format(idx), cell_year, cell_univ)
            df_excel = df_excel.drop(idx) #제목행 복사로 반복행 제거   
        
                     

        if cell_year == 'None' and cell_univ == 'None' and cell_type == 'None':
            _dbg_('{}행 삭제'.format(idx), cell_year, cell_univ, cell_type)
            df_excel = df_excel.drop(idx) #제목 3행중 2행 삭제
        #elif cell_year != 'None':
            #find_year = cell_year
            #_dbg_('찾은 년도 : {}'.format(find_year))
    
    df_excel = reset_index(df_excel)
    #print('찾은 년도 : {}'.format(find_year))
    
    
    header_info = df_excel.columns.tolist()
    _dbg_('header_info : ', type(header_info), header_info[:8], '...')

    
    return df_excel#, find_year
        

#################################################
# df_excel에서 [학교종류 | 설립구분 | 지역 | 상태] 열 삭제 -> university에서 통합 관리함
#################################################
def remove_univ_col(df_excel):
    #_dbg_()
    
    # 동일패턴A remove_univ_col
    df_header = list(df_excel.columns) #컬럼정보
    
    for hdr in df_header:
        if hdr == '설립구분':
            df_excel = df_excel.drop(['설립구분'], axis = 1)
        elif hdr == '학교종류':
            df_excel = df_excel.drop(['학교종류'], axis = 1)
        elif hdr == '지역':
            df_excel = df_excel.drop(['지역'], axis = 1)
        elif hdr == '상태':
            df_excel = df_excel.drop(['상태'], axis = 1)
        elif hdr == '상태정보':
            df_excel = df_excel.drop(['상태정보'], axis = 1)
        elif hdr == '법인명':
            df_excel = df_excel.drop(['법인명'], axis = 1)            

    ## 삭제 후 정리된 헤터 정보 출력
    df_info('( remove_univ_col ) df_excel 컬럼 삭제 완료 - ', df_excel)

    return df_excel
            
#################################################
# 연도 찾아서 채우기
#################################################
def fill_year_col(df_excel, find_year, col_name = '기준연도',):
    #_dbg_(df_excel.head())
    #_dbg_(find_year, col_name)
    
    #column_values = list(df_excel[col_name])
    #print(column_values)
        
    #df_excel = df_excel[col_name].fillna(cell, inplace=True)
    df_excel[col_name].replace('None', find_year, inplace=True)
    
    _dbg_('성공 : ', find_year, '년도 채우기')
    #_dbg_(df_excel.head(5))
    return df_excel


#################################################
# 삭제된 df index 재설정
#################################################
def reset_index(df):
    #삭제된 줄만큼 루프를 더 돌던지.... 인덱스 값을 재조정하던지
    df = df.reset_index(drop=True) #(inplace=True)
    return df
       

#################################################
# 병합된 셀들의 none값을 채우기
#################################################
def fill_none_col(df_excel, col_name = '학교명'):
    
    col_header = list(df_excel.columns) #헤더부분 컬럼정보
    _dbg_('col_header', col_header)
    row_idx = list(df_excel.index.values) #전체 행 수
    col_data = list(df_excel[col_name]) #헤더명에 일치하는 컬럼 데이터
    _dbg_('col_data', col_data)
    
    cur_cell = ''
    
    idx_max = len(row_idx)-1

    for idx in row_idx:
        #print(idx, idx_max)
        if idx >= idx_max:
            break
        
        cur_cell = df_excel.loc[idx, col_name]
        next_cell = df_excel.loc[idx+1, col_name]
        
        # if col_name == '기준연도':
        #     _dbg_(idx, cur_cell, next_cell)
        
        # None으로 된 셀들에 대해서만 작업.... 따라서 숫자 셀들은 영향 없음음
        #a.empty, a.bool(), a.item(), a.any() or a.all().
        
        if cur_cell != '' and next_cell == 'None':
            #_dbg_('[{}] 병합셀 입력({}) : cur_cell = {} , next_cell = {} '.format(col_name, cur_cell, cur_cell, next_cell))
            df_excel.loc[idx+1, col_name] = cur_cell

    return df_excel



#################################################
# 병합된 셀들 데이터 채우기
#################################################
def fill_merge_cell(df_excel):
    
    col_header = list(df_excel.columns) #전체 헤더 컬럼정보
    row_idx = list(df_excel.index.values) #전체 행 수
    #col_data = list(df_excel[col_name]) #헤더명에 일치하는 컬럼 데이터
    

    for col_cnt in range(len(col_header)):
        col_name = col_header[col_cnt]
        
        df_excel = fill_none_col(df_excel, col_name=col_name) # 'new-hdr-0'
        
        #col_data = list(df_excel[col_name]) #헤더명에 일치하는 컬럼 데이터
        _#dbg_(col_name, col_data)
        
    _dbg_('병합된 셀들에 None값 대체하기 완료')
    _dbg_(df_excel.head())
            
    return df_excel


#################################################
# 대학명에서 좌우공백 제거... 전체 공백을 지우는게 유리할수도...
#################################################
def strip_univ_name(df_excel, col_name='학교명'):
    
    for n in range(len(df_excel)): #0~491 데이터프레임에서 학교명을 코드명으로 변경
        df_excel_univname = str(df_excel.loc[n, col_name])
        
        #df_excel_univname = df_excel_univname.strip() 전체
        df_excel_univname = df_excel_univname.lstrip()
        df_excel_univname = df_excel_univname.rstrip()

    _dbg_('학교명 공백 제거 완료')
    return df_excel




#################################################
# 데이터 프레임의 학교명을 코드명으로 변경
# - 학교명과 일치하는 univ_cd를 찾아서 데이터프레임 정리
# conn, dbname, excel_file, sheet_name : university 테이블에 등록되지 않은 학교를 발견한 경우 다시 정보를 읽어오기 위함
# curs와 df_excel 만 실제로 필요함
#################################################
def set_univ_code(curs, conn, df_excel, dbname, template_file, excel_file, sheet_name):
    _#dbg_(excel_file, sheet_name)
    
    _dbg_('학교명을 코드명으로 변경 시작')

    add_new_univ_code = False #동일 대학명이 여러개 들어갈 수도 있기 때문에 DB에 등록된 정보를 다시 읽어와야 함

    sql_select = 'SELECT {}, {}, {} FROM {}'.format('univ_cd', 'univ_name_arlimi', 'univ_name', 'university') #university 테이블 기준으로 매핑하기 때문에 고정 값
    curs.execute(sql_select)

    univ_data = curs.fetchall() #university에 등록된 대학교 목록
    #_dbg_(type(univ_data), univ_data) # <class 'list'> [{'univ_cd': '0000001', 'univ_name_arlimi': '국립강릉원주대학교', 'univ_name': '강릉원주대학교1(강릉)'}, {'univ_cd': '0000002', 'univ_name_arlimi': '국립강릉원주대학교 _제2캠퍼스', 'univ_name': '강릉원주대학교2(원주)'},  ...
    
    # print(type(univ_data), univ_data)
    # print(type(univ_data[0]), univ_data[0])
    # print(type(univ_data[0]['univ_name_arlimi']), univ_data[0]['univ_name_arlimi'], univ_data[0]['univ_cd'])
    #conn.commit()

    #print('len(df_excel) : ' , len(df_excel))
    for n in range(len(df_excel)): #엑셀 데이터프레임에서 학교명을 추출

        df_excel_univname = str(df_excel.loc[n, '학교명'])
        #_dbg_('refine 전 : ', df_excel_univname)
        df_excel_univname = refine_univ_name(df_excel_univname)
        #_dbg_('refine 후 : ', df_excel_univname)
                    
        find_univ_code = False #university 테이블에 대학코드가 등록되지 않는 경우도 있음, 없다고 가정
        
        # A.엑셀대학명과 university 테이블의 코드명 매핑 - 대부분은 여기서 완료됨
        for i in range(len(univ_data)): #466 university 테이블에서 검색
            #if df_excel_univname == '화성의과학대학교':
            #print(n, df_excel_univname)

            univ_name_arlimi = univ_data[i]['univ_name_arlimi']
            univ_name_arlimi = refine_univ_name(univ_name_arlimi)
            
            # 일단은 사용하지 않음
            univ_name_general = univ_data[i]['univ_name']
            univ_name_general = refine_univ_name(univ_name_general)
            
            #_dbg_('univ_name_arlimi', univ_name_arlimi)
            #_dbg_('univ_name_general', univ_name_general)
            

            #(알리미 또는 univ_name명칭) 엑셀 학교명과 university 테이블에서 일치하는 학교명 발견하면 코드로 대체
            if df_excel_univname == univ_name_arlimi:
                #dat = '알리미 명칭 = df_excel_univname : {}, univ_name_arlimi : {}, univ_name_general : {}'.format(df_excel_univname, univ_name_arlimi, univ_name_general)
                #_dbg_(dat)
                #_dbg_('학교명 :', df_excel.loc[n, '학교명'], '학교코드 : ', univ_data[i]['univ_cd'])
                df_excel.loc[n, '학교명'] = univ_data[i]['univ_cd']
                find_univ_code = True
                break
            elif df_excel_univname == univ_name_general:
                dat = '일반 명칭 = df_excel_univname : {}, univ_name_arlimi : {}, univ_name_general : {}'.format(df_excel_univname, univ_name_arlimi, univ_name_general)
                _dbg_(dat)
                _dbg_('학교명 :', df_excel.loc[n, '학교명'], '학교코드 : ', univ_data[i]['univ_cd'])
                df_excel.loc[n, '학교명'] = univ_data[i]['univ_cd']                
                find_univ_code = True
                break

        
        # 아래 부분은 재작업하자... 24.02.16
        # universiti 테이블에 학교가 등록되지 않음 경우 -------------------어쩌다 한번씩 벌어질 일이기 때문에 그냥 재작업하자    경남정보..[폐교] 같은 케이스
        # 코드가 없는 학교명을 찾음
        # df_exce에서 없는 정보를 읽어옴
        # 신규 학교를 university 테이블에 등록
        # 신규 코드 부여
        if find_univ_code == False: #university 테이블에 등록되지 않은 학교를 발견, DB에 등록, ex)신경주대학교
            _dbg_('\n\n학교명과 일치하는 코드를 찾지 못함(신규 학교를 등록해야함) ', n, '번째 인덱스', df_excel_univname)
            
            
            #df_excel, doc_title, df_template_shape0 = compare_schema(template_file, excel_file, sheet_name) #get_table_name(doc_title) 에서 사용
            #df_excel, find_year = remove_empty_row1(df_excel, df_template_shape0)
            
            df_excel_add = pd.read_excel(excel_file, sheet_name = sheet_name)
            df_template_add = pd.read_excel(template_file, sheet_name = sheet_name)
            col_template, row_template = get_df_shape(df_template_add)
            #_dbg_('col_template', type(col_template), col_template)
            #_dbg_('row_template', type(row_template), row_template)
            
            df_excel_add = df_excel_add.fillna('None')
            df_excel_add, find_year = remove_empty_row1(df_excel_add, col_template) #
            df_excel_add = remove_empty_row2(df_excel_add)
            
            index_values = list(df_excel_add.index.values)
            for idx in index_values:
                #cell_year = df_excel.loc[idx, '기준연도']
                cell_univ_name = df_excel_add.loc[idx, '학교명']
                cell_univ_name = refine_univ_name(cell_univ_name)
                
                #_dbg_(idx, cell_univ_name)
                
                #신규 대학 발견, 관련 정보를 university 테이블에 등록
                if cell_univ_name == df_excel_univname:
                    #_dbg_(df_excel_add.head())
                    #_dbg_(idx)
                    _dbg_('신규 대학 발견, university 테이블에 등록해야 함 (행번호, 다시 오픈한 엑셀 대학명, 엑셀대학명)', idx, cell_univ_name, df_excel_univname)
                    
                    # 동일패턴A remove_univ_col
                    df_header = list(df_excel.columns) #컬럼정보
                    cell_estblish = ''
                    cell_type = ''
                    cell_locale = ''
                    cell_status = ''
                    for hdr in df_header:
                        if hdr == '설립구분':
                            cell_estblish = df_excel_add.loc[idx, '설립구분']
                            _dbg_('cell_estblish : ', cell_estblish)
                        elif hdr == '학교종류':
                            cell_type = df_excel_add.loc[idx, '학교종류']
                            _dbg_('cell_type : ', cell_type)
                        elif hdr == '지역':
                            cell_locale = df_excel_add.loc[idx, '지역']
                            _dbg_('cell_locale : ', cell_locale)
                        elif hdr == '상태':
                            cell_status = df_excel_add.loc[idx, '상태']
                            _dbg_('cell_status : ', cell_status)
                        elif hdr == '상태정보':
                            cell_status = df_excel_add.loc[idx, '상태정보']
                            _dbg_('cell_status : ', cell_status)
            
                    
                    #university 테이블의 코드값중 최대값을 찾음
                    max_univ_code = get_univ_maxcode(curs)
                    
                    _dbg_('등록할 코드 [{}] : 학교종류 ({}), 설립구분 ({}), 상태 ({}), 지역 ({})'.format(max_univ_code, cell_type, cell_estblish, cell_status, cell_locale))

                    #'insert into cosshubtest.university(univ_cd, univ_name_arlimi, univ_name, univ_name_en, univ_type, establish, status, campus, campus_name, locale, homepage, tel, fax) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
                    
                    #col_name_str = '`univ_cd`, `univ_name_arlimi`, `univ_type`, `scale`, `locale`, `establish`, `univ_name`, `univ_name_en`, `campus`, `campus_name`, `corporate_name`, `status`, `founding`, `closing`, `homepage`, `tel`, `fax`, `remark`'
                    col_name_str = '`univ_cd`, `univ_name_arlimi`, `univ_type`, `locale`, `establish`,`univ_name`, `status`, `remark`'
                    #field_data_str = "'{}', '{}', '{}', 'None', '{}', '{}', '{}', 'None', 'None', '{}', 'None', 'None', 'None'".format(max_univ_code, cell_univ_name, cell_type, cell_univ_name, cell_estblish, cell_status, cell_locale)
                    field_data_str = "'{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}'".format(max_univ_code, cell_univ_name, cell_type, cell_locale, cell_estblish, cell_univ_name, cell_status, '신규등록')
                    sql_new_univ = 'insert into {}.{}({}) values({})'.format(dbname, 'university', col_name_str, field_data_str)
                    
                    curs.execute(sql_new_univ)
                    conn.commit()
                    
                    #동일 대학명이 여러개 들어갈 수도 있기 때문에 DB에 등록된 정보를 다시 읽어와야 함
                    #sql_select = 'SELECT {}, {} FROM {}'.format('univ_cd', 'univ_name_arlimi', 'university') #university 테이블 기준으로 매핑하기 때문에 고정 값
                    sql_select = 'SELECT {}, {}, {} FROM {}'.format('univ_cd', 'univ_name_arlimi', 'univ_name', 'university') #university 테이블 기준으로 매핑하기 때문에 고정 값
                    _dbg_(sql_select)
                    curs.execute(sql_select)
                    univ_data = curs.fetchall() #university에 등록된 대학교 목록
                    
                    
                    #엑셀 학교명과 university 테이블에서 일치하는 학교명 발견하면 코드로 대체
                    #if df_excel_univname == univ_data[i]['univ_name_arlimi']:
                        #print(univ_data[i]['univ_name_arlimi'], univ_data[i]['univ_cd'])
                        
                    df_excel.loc[n, '학교명'] = max_univ_code
            
    return df_excel



#################################################
# 기준이 되는 학교정보 테이블 입력, 외부데이터와는 달리 자체 생성 데이터
#################################################
def create_university(conn, curs, dbname, table, excel_path, sheet_name, table_reset = False, mode_sql = 'I/U/D'):
    _dbg_(table)
    df = pd.read_excel(excel_path, sheet_name = sheet_name)

    # 데이터프레임의 nan 값을 None으로 변경
    #df.dropna(inplace=True)
    #df = df.where((pd.notnull(df)), 'None')
    df = df.fillna('None')

    #df.rename(columns={'대학 고유코드':'a'}, inplace=True) #특정 컬럼명 변경
    #df.columns('a', 'b') 

    cols_ko = list(df.columns) #컬럼 시리즈를 리스트로 변경
    cols_en = list(df.iloc[0]) #데이터 시리즈를 리스트로 변경

    for n in range(len(cols_ko)):
        #print(cols_en[n], cols_ko[n])
        df.rename(columns={cols_ko[n]:cols_en[n]}, inplace=True) #특정 컬럼명 변경
    df = df.drop(0) #특정행 삭제
    
    # db truncate tables, 기존 데이터를 삭제 후 인서트
    if table_reset == True:
        sql_truncate = 'truncate table {}.{}'.format(dbname, table)
        curs.execute(sql_truncate)
        conn.commit()
        print('{}.{} 테이블 삭제 완료'.format(dbname, table), sql_truncate)
    
    # data insert to db
    field_names = ''
    field_data = ''


    for n in range(len(cols_en)):
        if n == 0 :
            field_names = cols_en[n]
            field_data = '%s'
        else:
            field_names = field_names + ', ' + cols_en[n]
            field_data = field_data + ', ' + '%s'
            
        #입력데이터 중 좌우측 공백제거
        field_data = field_data.lstrip()
        field_data = field_data.rstrip()
        
        
    # sql_insert = 'insert into cosshubtest.university(univ_cd, univ_name_arlimi, univ_name, univ_name_en, univ_type, establish, status, campus, campus_name, locale, homepage, tel, fax) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
    # print(sql_insert)

    univ_name_arlimi_date = ''
    univ_cd=''
    if mode_sql == 'I': #insert mode
        sql = 'insert into {}.{}({}) values({})'.format(dbname, table, field_names, field_data)
    elif mode_sql == 'U': #update mode, 알리미 대학명 이름만 변경함 기준은 대학코드 >>>> 변경된 데이터만 찾아서 향후 별도 처리
        sql = 'update {}.{} set {}={} where univ_cd={}'.format(dbname, table, 'univ_name_arlimi', univ_name_arlimi_date, univ_cd)
    
    print('1번 : ', sql)
    
    for idx in range(len(df)):
        #print(idx, sql)
        curs.execute(sql, tuple(df.values[idx]))
    conn.commit()
    
    if mode_sql == 'I': #insert mode
        print(idx, '개의 데이터 입력 성공')
    elif mode_sql == 'U': #update mode, 알리미 대학명 이름만 변경함 기준은 대학코드
        print(idx, '개의 데이터 업데이트 성공')
    
    # UPDATE cosshubtest.university
	# SET univ_name_arlimi='None-'
	# WHERE univ_cd='0002653'
 
 
 


In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
###################################################################################################
# 학과정보 엑셀을 읽어서 DB에 입력
###################################################################################################

#!pip install sqlalchemy

import pandas as pd
import pymysql
import sqlalchemy
from sqlalchemy import create_engine

# 엑셀 파일을 읽어서 DB 테이블에 데이터 입력
def excel2table(conn, excel, sheet, table, opt='replace'):
    
    #df = pd.read_excel(excel, sheet, dtype={'id': str})
    df = pd.read_excel(excel, sheet, dtype=str) #문자열로 읽기
    _dbg_(df.head())

    
    user = 'root'
    password = 's!W@u#12' #비번에 @가 들어가서 문제가 생김
    host = '13.124.114.176'
    db = 'swu_external'
    port = 3306
    
    #비번에 @가 들어가서 문제가 생기면 이렇게 처리
    url = sqlalchemy.engine.URL.create(
        drivername="mysql+pymysql",
        username=user,
        password=password,
        host=host,
        port=port,
        database=db,
    )

    # 데이터베이스 연결
    #conn, curs = db_setting('13.124.114.176', 'root', 's!W@u#12', dbname = dbname)
    #db_connection_str = 'mysql+pymysql://root:{}@13.124.114.176/swu_external'.format('s!W@u#12')
    db_connection_str = f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}'
    engine = create_engine(url)

    # https://docs.kanaries.net/ko/topics/Pandas/pandas-to-sql
    # name: SQL 테이블 이름 (필수)
    # con: sqlite 또는 sqlalchemy에서 제공하는 연결 (필수)
    # schema: SQL 데이터베이스 스키마
    # if_exists: 테이블이 이미 존재하는 경우 수행할 작업, 'fail', 'replace' 또는 'append' 중 하나
    # index: DataFrame의 인덱스 열을 테이블에 작성할 지 여부
    # index_label: 인덱스 열의 이름 (테이블에 작성되는 경우)
    # chunksize: 한 번에 작성할 행 수
    # dtype: 모든 열과 해당 데이터 유형의 사전
    # method: SQL 삽입 문을 지정


    판다스로 읽을때 엑셀 기본위에 5줄정도는 지우고 읽도록하자
    엑셀의 제목행이 테이블의 컬럼명으로 지정된다.

    # 데이터 삽입
    df.to_sql(table, con=engine, if_exists=opt, index=False)
    print("데이터 삽입 완료!")



In [95]:
dbname = 'swu_external'

#conn, curs = db_setting('133.186.247.101', 'root', 'coss!@#$', dbname = 'cosshubtest') #coss test
conn, curs = db_setting('13.124.114.176', 'root', 's!W@u#12', dbname = dbname)


excel = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/11.학과분류체계/0.학과 표준분류 통합.xlsx'
sheet_name = '학과표준분류체계'

table = 'dep_codes'

excel2table(conn, excel, sheet_name, table, 'replace')

[dbg] ( excel2table )    seq class_name_1 class_code_1 class_name_2 class_code_2 class_name_3  \
0   1        인문ㆍ사회            A        언어ㆍ문학           01          언어학   
1   2        인문ㆍ사회            A        언어ㆍ문학           01       국어ㆍ국문학   
2   3        인문ㆍ사회            A        언어ㆍ문학           01       독일어ㆍ문학   
3   4        인문ㆍ사회            A        언어ㆍ문학           01      러시아어ㆍ문학   
4   5        인문ㆍ사회            A        언어ㆍ문학           01      스페인어ㆍ문학   

  class_code_3 remark  
0          001    NaN  
1          002    NaN  
2          003    NaN  
3          004    NaN  
4          005    NaN       
데이터 삽입 완료!


In [ ]:
sys.exit(1)

In [ ]:
import sys, os

#file = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료'
#sys.path.append('D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료')

#from tables_schema import *
#from tables_schema import tables_schema
#import tables_schema


#################################################
# 기준년도 + 학교코드가 키 값이 아니라 idx기준을 데이터 인서트
#################################################
def insert_df(curs, conn, df_excel, dbname, table_name, mode = 'create'):
    _dbg_(table_name, '에 데이터 입력 시작')
    cnt_insert = 0
    cnt_update = 0

    #DB 테이블의 헤더정보만 추출    
    col_info, col_name_org, col_name_list = get_db_column_name(curs, dbname, table_name) #리턴값중 col_info만 사용

    # 테스트코드 데이터 타입만 읽어오기    
    # data_type = get_data_type(col_info, 'year')
    # _dbg_(data_type)
    
    #_dbg_('col_info : ', len(col_info), col_info)
    #_dbg_('col_name_list : ', len(col_name_list), col_name_list)
    
    for i in range(len(col_info)):
        _dbg_('작업 컬럼명 ', i, ' = ', col_info[i]['COLUMN_NAME'])
    #sys.exit()
    
    # reg_date와 mod_date는 처리에서 제외 > 241007 테이블에 신규 컬럽 추가(계약학과 설치 운영현황, new_tech) 후 문제가 있어서 아래 루프 코드에서 처리함
    col_info_cnt = len(col_info)
    # for i in range(len(col_info)):
    #     if col_info[i]['COLUMN_NAME'] == 'reg_date' and col_info[i]['DATA_TYPE'] == 'datetime':
    #         col_info_cnt = col_info_cnt - 1
    #     elif col_info[i]['COLUMN_NAME'] == 'mod_date' and col_info[i]['DATA_TYPE'] == 'datetime':
    #         col_info_cnt = col_info_cnt - 1
            
    # print(df_excel.head(15))
    # sys.exit(1)
    
    _dbg_('전체 작업 대상 컬럼 수(테이블) : ', col_info_cnt)
    _dbg_('전체 작업 대상 레코드 수(엑셀) : ', len(df_excel))

    #df_excel 데이터프레임에서 읽어온 데이터와 테이블에 있는 데이터를 비교해서 변경된 사항은 업데이트
    for n in range(len(df_excel)): #전처리 완료된 데이터 프레임 전체를 읽어서 DB에 업데이트

        field_data = list(df_excel.iloc[n]) # 시리즈 -> 리스트
        _dbg_('입력할 행 data : [', n, '] ', field_data)
        
        first_field = True #sql생성을 위해 처음과 중간 끝 작업을 위해
        last_field = False #마지막 컬럼 처리했는지 확인
        idx_cnt = 0 #idx는 자동증가이기 때문에 처리하지 않음 따라서 field_data[m]과 -1개 차이가 발생함
        col_name_str = ''
        col_data_str = ''
        select_where = ''
        update_data = ''
        
        
        for m in range(col_info_cnt):
            
            #idx는 자동증가이므로 제외
            col_name = col_info[m]['COLUMN_NAME']
            col_type = col_info[m]['DATA_TYPE']
            
            #_dbg_(col_name, col_type, field_data[m-idx_cnt])
            
            #_dbg_('col_name + col_type : ', col_name, col_type)
            if(col_name == 'idx' or col_name == 'reg_date' or col_name == 'mod_date'):
                idx_cnt += 1
                _dbg_("idx mode ON - pass column", idx_cnt)
                
            else: #idx이외의 나머지 필드
                
                if first_field == True:
                    #_dbg_('처음({}/{}), col_name:{}, col_type:{}'.format(m, col_info_cnt, col_name, col_type))  #idx가 있으면 하나 빼야함!!
                    col_name_str = '`{}`, '.format(col_name) #처음
                    
                    _dbg_('첫열 데이터({}/{})'.format(m, col_info_cnt), col_name, col_type, field_data[m-idx_cnt])
                    if col_type == 'int':
                        #_dbg_('int', col_name, col_type, field_data[m-idx_cnt])
                        norm_val = norm_value(field_data[m-idx_cnt], ',-') #, 제거(처음)
                        col_data_str = '{}, '.format(norm_val)
                        select_where = '`{}` = {} and '.format(col_name, norm_val)
                        
                    elif col_type == 'float':
                        #_dbg_('float', col_name, col_type, field_data[m-idx_cnt])
                        norm_val = norm_value(field_data[m-idx_cnt], ',-') #, 제거(처음)
                        col_data_str = '{}, '.format(norm_val)
                        select_where = '`{}` = {} and '.format(col_name, norm_val)
                        
                    elif col_type == 'decimal':
                        #_dbg_('bigint', col_name, col_type, field_data[m-idx_cnt])
                        norm_val = norm_value(field_data[m-idx_cnt], ',-') #, 제거(처음)
                        col_data_str = '{}, '.format(norm_val)
                        select_where = '`{}` = {} and '.format(col_name, norm_val) 

                    elif col_type == 'bigint':
                        #_dbg_('bigint', col_name, col_type, field_data[m-idx_cnt])
                        norm_val = norm_value(field_data[m-idx_cnt], ',-') #varchar 타입 . , 등은 허용(처음)
                        col_data_str = '{}, '.format(norm_val)
                        select_where = '`{}` = {} and '.format(col_name, norm_val) 
                        
                    elif col_type == 'varchar' or col_type == 'longtext':
                        #_dbg_('varchar', col_name, col_type, field_data[m-idx_cnt])
                        norm_varchar = norm_value(field_data[m-idx_cnt], '"') #varchar 타입 . , 등은 허용(처음)
                        col_data_str = '"{}", '.format(norm_varchar)
                        select_where = '`{}` = "{}" and '.format(col_name, norm_varchar)
                        
                    first_field = False
                    

                ##elif m == col_info_cnt-(idx_cnt+2): #끝 - 최초 idx_cnt는 0에서 발견된 reg_date와 mod_date는 추가해야 함, reg_date와 mod_date 필드가 반드시 있어야함
                elif (m == col_info_cnt-(idx_cnt+3)) and (last_field == False): #끝 - 최초 idx_cnt는 0에서 발견된 reg_date와 mod_date는 추가해야 함, reg_date와 mod_date 필드가 반드시 있어야함
                    #_dbg_('끝({}/{}), col_name:{}, col_type:{}'.format(m, col_info_cnt, col_name, col_type))
                    col_name_str = col_name_str + '`{}` '.format(col_name) #마지막
                    
                    # _dbg_('idx_cnt = ', idx_cnt)
                    # _dbg_('m, col_info_cnt, col_name, col_type = ', m, col_info_cnt, col_name, col_type)
                    # _dbg_('field_data[m-idx_cnt-1] = ', field_data[m-idx_cnt-1])
                    
                    _dbg_('끝열 데이터({}/{})'.format(m, col_info_cnt), col_name, col_type, field_data[m-idx_cnt-1])
                    if col_type == 'int':
                        norm_val = norm_value(field_data[m-idx_cnt], ',-') #, 제거(마지막)
                        col_data_str = col_data_str + '{}'.format(norm_val)
                        select_where = select_where + '`{}` = {}'.format(col_name, norm_val)
                        
                    elif col_type == 'float':
                        norm_val = norm_value(field_data[m-idx_cnt], ',-') #, 제거(마지막)
                        col_data_str = col_data_str + '{}'.format(norm_val)
                        select_where = select_where + '`{}` = {}'.format(col_name, norm_val)
                        
                    elif col_type == 'decimal':
                        norm_val = norm_value(field_data[m-idx_cnt], ',-') #, 제거(마지막)
                        col_data_str = col_data_str + '{}'.format(norm_val)
                        select_where = select_where + '`{}` = {}'.format(col_name, norm_val)

                    elif col_type == 'bigint':
                        #_dbg_('bigint', col_name, col_type, field_data[m-idx_cnt])
                        norm_val = norm_value(field_data[m-idx_cnt], ',-') #varchar 타입 . , 등은 허용(처음)
                        col_data_str = col_data_str + '{}'.format(norm_val)
                        select_where = select_where + '`{}` = {}'.format(col_name, norm_val)
                        
                    elif col_type == 'varchar' or col_type == 'longtext':
                        norm_varchar = norm_value(field_data[m-idx_cnt], '"') #varchar 타입 . , 등은 허용(마지막)
                        col_data_str = col_data_str + '"{}"'.format(norm_varchar)
                        select_where = select_where + '`{}` = "{}"'.format(col_name, norm_varchar)
                    
                    last_field = True
                    #_dbg_(norm_val)

                elif (first_field == False) and (last_field == False):
                    #_dbg_('중간({}/{}), col_name:{}, col_type:{}'.format(m, col_info_cnt, col_name, col_type))
                    col_name_str = col_name_str + '`{}`, '.format(col_name) #중간
                    norm_val = norm_value(field_data[m-idx_cnt], ',') #, 제거
                    
                    #_dbg_('중간열 데이터({}/{})'.format(m, col_info_cnt), col_name, col_type, field_data[m-idx_cnt])
                    if col_type == 'int':
                        norm_val = norm_value(field_data[m-idx_cnt], ',-') #, 제거(중간)
                        col_data_str = col_data_str + '{}, '.format(norm_val)
                        select_where = select_where + '`{}` = {} and '.format(col_name, norm_val)
                        
                    elif col_type == 'float':
                        norm_val = norm_value(field_data[m-idx_cnt], ',-') #, 제거(중간)
                        col_data_str = col_data_str + '{}, '.format(norm_val)
                        select_where = select_where + '`{}` = {} and '.format(col_name, norm_val)
                        
                    elif col_type == 'decimal':
                        norm_val = norm_value(field_data[m-idx_cnt], ',-') #, 제거(중간)
                        col_data_str = col_data_str + '{}, '.format(norm_val)
                        select_where = select_where + '`{}` = {} and '.format(col_name, norm_val)
                        
                    elif col_type == 'bigint':
                        #_dbg_('bigint', col_name, col_type, field_data[m-idx_cnt])
                        norm_val = norm_value(field_data[m-idx_cnt], ',-') #varchar 타입 . , 등은 허용(처음)
                        col_data_str = col_data_str + '{}, '.format(norm_val)
                        select_where = select_where + '`{}` = {} and '.format(col_name, norm_val)                
                        
                    elif col_type == 'varchar' or col_type == 'longtext':
                        norm_varchar = norm_value(field_data[m-idx_cnt], '"') ##varchar 타입 . , 등은 허용(중간)
                        col_data_str = col_data_str + '"{}", '.format(norm_varchar)
                        select_where = select_where + '`{}` = "{}" and '.format(col_name, norm_varchar)
                        
                        #_dbg_('중간(varchar) : ', norm_val)
                        
                #_dbg_('정제 중 입력할 col_data_str : ', col_data_str)
                #_dbg_('정제 중 입력할 select_where : ', select_where)
                
                #_dbg_('idx_cnt:{}, m:{}'.format(idx_cnt, m))
                
        if (first_field == True):
            _dbg_('첫 컬럼 처리에 오류 발생!', idx_cnt)
            sys.exit()
        elif (last_field == False):
            _dbg_('마지막 컬럼 처리에 오류 발생!', idx_cnt)
            sys.exit()
            

                                        
        _dbg_('정제 후 입력할 col_data_str : ', col_data_str)
        #_dbg_('입력할 norm_val : ', norm_val)

        #기존 테이블이 존재하면 업데이트 모드로 해야함
        #테이블은 존재하지만 실제 데이터는 없을수도 있음
        #테이블은 존재하지만 데이터가 업데이트가 된것들이 많음
        if mode == 'exist':
            sql = 'select * from {}.{} where {}'.format(dbname, table_name, select_where) #table_dic['전공과목 성적 분포'] = 'major_grades' 만 해당
            #_dbg_(select_where)
            _dbg_('exist mode: ' , sql)
            #_dbg_(col_data_str)
            #_dbg_(field_data)
            
            cnt = curs.execute(sql) #검색된 갯수
            result = curs.fetchall() #리스트 안에 딕셔너리로...
            #print(type(cnt), cnt, type(result), result)

            ### Ryu, 업데이트시 키값만 정확하게 읽어오는 거 필요 >> 다중키일경우 키 값만 정확하게 확인하는게 필요
            if cnt == 1: #정상 검색 -> 업데이트 대상    >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 키값 기준으로 검색을 해서 찾아야하는데 idx 모드일때 등 키값을 정확히 찾기가 어려움... 천천히 생각
                #print(cnt, '쿼리 검색 성공(업데이트) ')
                #if table == 'major_grades': #테이블로 구분하거나 semester로 구분해도 됨
                #sql = 'update {}.{} set {} where {}'.format(dbname, table, col_name_str, select_where)
                #_dbg_(cnt, '업데이트를 하면 좋은데 키값 기준에 대한 고민이 필요')
                #_dbg_(cnt, '기존 데이터 있음(업데이트 실행) : ', sql)
                # else:
                #     sql = 'update {}.{} set {} where year = {} and univ_cd = "{}"'.format(dbname, table, set_data, year, univ_cd)
                cnt_update += 1
            elif cnt == 0: #테이블은 존재하지만 데이터는 없는 경우이기 때문에 신규 인서트를 해야함
                sql = 'insert into {}.{}({}) values({})'.format(dbname, table_name, col_name_str, col_data_str)
                _dbg_('테이블 O, 데이터 X.... : ', sql)
                if sql != '':
                    curs.execute(sql)
                    
                cnt_insert += 1
            else:
                _dbg_(cnt, '오류 : 테이블이 존재하지만 알수 없는 오류 : ', sql)
                sys.exit(0)

        elif mode == 'create': 
            sql = 'insert into {}.{}({}) values({})'.format(dbname, table_name, col_name_str, col_data_str)
            _dbg_('신규 인서트 모드 : ', sql)
            cnt_insert += 1
            
            if sql != '':
                curs.execute(sql)
            #conn.commit()        
            
        else:
            _dbg_(cnt, '오류 : 정의되지 않음 ', sql)
            sys.exit(0)
        
    #     if sql != '':
    #         curs.execute(sql)
    conn.commit()
    
    print(table_name, '테이블 : ', cnt_insert, '개의 신규 인서트     ', cnt_update, '개의 업데이트')
    
    
    
    
#################################################
# 업데이트 모드는 키값 포함 신규데이터로 무조건 업데이트를 수행
#################################################
def update_df(curs, conn, df_excel, dbname, table_name, mode = 'exist'):
    _dbg_(table_name, '에 데이터 업데이트 시작')
    cnt_update = 0
    cnt_insert = 0

    #DB 테이블의 헤더정보만 추출
    col_info, col_name_org, col_name_list = get_db_column_name(curs, dbname, table_name) #리턴값중 col_info만 사용

    # 테스트코드 데이터 타입만 읽어오기    
    # data_type = get_data_type(col_info, 'year')
    # _dbg_(data_type)
    
    #_dbg_('col_info : ', len(col_info), col_info)
    #_dbg_('col_name_list : ', len(col_name_list), col_name_list)
    
    for i in range(len(col_info)):
        _dbg_('작업 컬럼명 ', i, ' = ', col_info[i]['COLUMN_NAME'])
    #sys.exit()
    
    # reg_date와 mod_date는 처리에서 제외 > 241007 테이블에 신규 컬럽 추가(계약학과 설치 운영현황, new_tech) 후 문제가 있어서 아래 루프 코드에서 처리함
    col_info_cnt = len(col_info)
    # for i in range(len(col_info)):
    #     if col_info[i]['COLUMN_NAME'] == 'reg_date' and col_info[i]['DATA_TYPE'] == 'datetime':
    #         col_info_cnt = col_info_cnt - 1
    #     elif col_info[i]['COLUMN_NAME'] == 'mod_date' and col_info[i]['DATA_TYPE'] == 'datetime':
    #         col_info_cnt = col_info_cnt - 1
            
    # print(df_excel.head(15))
    # sys.exit(1)
    
    primary_keys = get_db_primary_keys(curs, dbname, table_name)
    _dbg_(type(primary_keys), primary_keys)
    
    
    _dbg_('전체 작업 대상 컬럼 수(테이블) : ', col_info_cnt)
    _dbg_('전체 작업 대상 레코드 수(엑셀) : ', len(df_excel))

    #df_excel 데이터프레임에서 읽어온 데이터와 테이블에 있는 데이터를 비교해서 변경된 사항은 업데이트
    for n in range(len(df_excel)): #전처리 완료된 데이터 프레임 전체를 읽어서 DB에 업데이트

        field_data = list(df_excel.iloc[n]) # 시리즈 -> 리스트
        _dbg_('입력할 행 data : [', n, '] ', field_data)
        
        first_field = True #sql생성을 위해 처음과 중간 끝 작업을 위해
        last_field = False #마지막 컬럼 처리했는지 확인
        idx_cnt = 0 #idx는 자동증가이기 때문에 처리하지 않음 따라서 field_data[m]과 -1개 차이가 발생함
        col_name_str = ''
        col_data_str = ''
        select_where = ''
        update_data = ''
        
        
        for m in range(col_info_cnt):
            
            #idx는 자동증가이므로 제외
            col_name = col_info[m]['COLUMN_NAME']
            col_type = col_info[m]['DATA_TYPE']
            
            #_dbg_(col_name, col_type, field_data[m-idx_cnt])
            
            #_dbg_('col_name + col_type : ', col_name, col_type)
            if(col_name == 'idx' or col_name == 'reg_date' or col_name == 'mod_date'):
            #if(col_name == 'idx' or col_name == 'reg_date'):
                idx_cnt += 1
                _dbg_("idx mode ON - pass column", idx_cnt)
                
            else: #idx이외의 나머지 필드
                
                if first_field == True:
                    _dbg_('처음({}/{}), col_name:{}, col_type:{}'.format(m, col_info_cnt, col_name, col_type))  #idx가 있으면 하나 빼야함!!
                    col_name_str = '`{}`, '.format(col_name) #처음
                    
                    _dbg_('첫열 데이터({}/{})'.format(m, col_info_cnt), col_name, col_type, field_data[m-idx_cnt])
                            
                    if col_type == 'varchar' or col_type == 'longtext':
                        #_dbg_('varchar', col_name, col_type, field_data[m-idx_cnt])
                        norm_varchar = norm_value(field_data[m-idx_cnt], '"') #varchar 타입 . , 등은 허용(처음)
                        col_data_str_up = '{} = "{}", '.format(col_name, norm_varchar) #update에 사용
                        col_data_str_in = '"{}", '.format(norm_varchar) #insert에 사용
                        
                        if col_name in primary_keys:
                            select_where = '`{}` = "{}" and '.format(col_name, norm_varchar)
                    else: # 'float', 'decimal', 'bigint', 'int'
                        #_dbg_('int', col_name, col_type, field_data[m-idx_cnt])
                        norm_val = norm_value(field_data[m-idx_cnt], ',-') #, 제거(처음)
                        col_data_str_up = '{} = {}, '.format(col_name, norm_val) #update에 사용
                        col_data_str_in = '{}, '.format(norm_val) #insert에 사용
                        if col_name in primary_keys:
                            select_where = '`{}` = {} and '.format(col_name, norm_val)                            
                        
                    first_field = False
                    

                ##elif m == col_info_cnt-(idx_cnt+2): #끝 - 최초 idx_cnt는 0에서 발견된 reg_date와 mod_date는 추가해야 함, reg_date와 mod_date 필드가 반드시 있어야함
                elif (m == col_info_cnt-(idx_cnt+3)) and (last_field == False): #끝 - 최초 idx_cnt는 0에서 발견된 reg_date와 mod_date는 추가해야 함, reg_date와 mod_date 필드가 반드시 있어야함
                    _dbg_('끝({}/{}), col_name:{}, col_type:{}'.format(m, col_info_cnt, col_name, col_type))
                    col_name_str = col_name_str + '`{}` '.format(col_name) #마지막
                    
                    # _dbg_('idx_cnt = ', idx_cnt)
                    # _dbg_('m, col_info_cnt, col_name, col_type = ', m, col_info_cnt, col_name, col_type)
                    # _dbg_('field_data[m-idx_cnt-1] = ', field_data[m-idx_cnt-1])
                    
                    _dbg_('끝열 데이터({}/{})'.format(m, col_info_cnt), col_name, col_type, field_data[m-idx_cnt-1])
                    
                    if col_type == 'varchar' or col_type == 'longtext':
                        norm_varchar = norm_value(field_data[m-idx_cnt], '"') #varchar 타입 . , 등은 허용(마지막)
                        col_data_str_up = col_data_str_up + '{} = "{}"'.format(col_name, norm_varchar) #update에 사용
                        col_data_str_in = col_data_str_in + '"{}"'.format(norm_varchar) #insert에 사용
                        if col_name in primary_keys:
                            select_where = select_where + '`{}` = "{}"'.format(col_name, norm_varchar)                    
                    else: # 'float', 'decimal', 'bigint', 'int'
                        norm_val = norm_value(field_data[m-idx_cnt], ',-') #, 제거(마지막)
                        col_data_str_up = col_data_str_up + '{} = {}'.format(col_name, norm_val) #update에 사용
                        col_data_str_in = col_data_str_in + '{}'.format(norm_val) #insert에 사용
                        if col_name in primary_keys:
                            select_where = select_where + '`{}` = {}'.format(col_name, norm_val)
                    
                    last_field = True
                    #_dbg_(norm_val)

                elif (first_field == False) and (last_field == False):
                    _dbg_('중간({}/{}), col_name:{}, col_type:{}'.format(m, col_info_cnt, col_name, col_type))
                    col_name_str = col_name_str + '`{}`, '.format(col_name) #중간
                    norm_val = norm_value(field_data[m-idx_cnt], ',') #, 제거
                    
                    if col_type == 'varchar' or col_type == 'longtext':
                        norm_varchar = norm_value(field_data[m-idx_cnt], '"') ##varchar 타입 . , 등은 허용(중간)
                        col_data_str_up = col_data_str_up + '{} = "{}", '.format(col_name, norm_varchar) #update에 사용
                        col_data_str_in = col_data_str_in + '"{}", '.format(norm_varchar) #insert에 사용
                        if col_name in primary_keys:
                            select_where = select_where + '`{}` = "{}" and '.format(col_name, norm_varchar)
                    else: # 'float', 'decimal', 'bigint', 'int'
                        norm_val = norm_value(field_data[m-idx_cnt], ',-') #, 제거(중간)
                        col_data_str_up = col_data_str_up + '{} = {}, '.format(col_name, norm_val) #update에 사용
                        col_data_str_in = col_data_str_in + '{}, '.format(norm_val) #insert에 사용
                        if col_name in primary_keys:
                            select_where = select_where + '`{}` = {} and '.format(col_name, norm_val)                        
                        
                        #_dbg_('중간(varchar) : ', norm_val)
                        
                #_dbg_('정제 중 입력할 col_data_str_up : ', col_data_str_up)
                #_dbg_('정제 중 입력할 select_where : ', select_where)
                
                #_dbg_('idx_cnt:{}, m:{}'.format(idx_cnt, m))
                
        if (first_field == True):
            _dbg_('첫 컬럼 처리에 오류 발생!', idx_cnt)
            sys.exit()
        elif (last_field == False):
            _dbg_('마지막 컬럼 처리에 오류 발생!', idx_cnt)
            sys.exit()
            
        
        _dbg_("'", select_where[-4:], "'")
        select_where = select_where.rstrip()
        if select_where[-4:] == ' and':
            select_where = select_where[:-4]
            
        col_data_str_up = col_data_str_up + ', mod_date = {}'.format('current_timestamp()')

        _dbg_('정제 후 입력할 col_name_str : ', col_name_str)
        _dbg_('정제 후 입력할 col_data_str_up : ', col_data_str_up)
        _dbg_('정제 후 입력할 col_data_str_in : ', col_data_str_in)
        _dbg_('정제 후 입력할 키 select_where : ', select_where)            
        
        
        
        #sys.exit()
        #_dbg_('입력할 norm_val : ', norm_val)
        
        
        # 데이터를 업데이트할지 인서트를 할지 판별 필요
        sql = 'select * from {}.{} where {}'.format(dbname, table_name, select_where) #프라이머리 키값과 일치하는 정보가 있는지 확인, 있으면 없데이트, 없으면 인서트
        cnt = curs.execute(sql) #검색된 갯수
        _dbg_("update할지 insert할지 판단 : ", cnt)
        if cnt > 0:
            mode = 'update'
        else:
            mode = 'insert'
        
        # 존재하는 테이블에 기존 입력된데이터가 있는 경우
        if mode == 'update': 
            #sql = 'select * from {}.{} where {}'.format(dbname, table_name, select_where) #table_dic['전공과목 성적 분포'] = 'major_grades' 만 해당
            sql = 'update {}.{} set {} where {}'.format(dbname, table_name, col_data_str_up, select_where)
            #_dbg_(select_where)
            _dbg_('기존 테이블에 업데이트 모드 : ', sql)
            #_dbg_(col_data_str_up)
            #_dbg_(field_data)
            
            cnt = curs.execute(sql) #검색된 갯수
            result = curs.fetchall() #리스트 안에 딕셔너리로...
            cnt_update += 1
            #_dbg_('select 수, 업데이트 결과 : ', cnt, result)

        # 존재하는 테이블에 입력된데이터가 없는 경우
        elif mode == 'insert': 
            sql = 'insert into {}.{}({}) values({})'.format(dbname, table_name, col_name_str, col_data_str_in)
            _dbg_('기존 테이블에 인서트 모드 : ', sql)
            
            cnt = curs.execute(sql)
            result = curs.fetchall() #리스트 안에 딕셔너리로...
            cnt_insert += 1
            
        else:
            _dbg_('업데이트 오류 : 정의되지 않음 ', sql)
            sys.exit(0)
    
    conn.commit()
    
    print(table_name, '테이블 : ', cnt_update, '개의 업데이트', cnt_insert, '개의 인서트')

In [ ]:
##################################################################################################
##################################################################################################
#메인 작업
##################################################################################################
##################################################################################################

#241118 21~24 전체 업데이트
#교육여건
# 2022년 _대학_3-나. 전임 입학사정관 현황_학교별자료
# 2022년 _대학_12-차. 공동활용 연구장비 운영 현황_학교별자료
# 2022년 _대학_13-나. 대학 부설 연구소 현황_학교별자료
# 2022년 _대학_3-다. 학교생활기록부의 기록과 자기소개서 등 교과성적 외의 자료를 활용하는 입학전형에서 서류평가를 담당한 평가자 1명당 서류평가 건수_학교별자료
# 2022년 _대학_14-다. 교지(校地) 확보 현황_학교별자료
# 2022년 _대학_6-다-2. 전공계열별 외국인 전임교원 현황_학교별자료
# 2022년 _대학_6-다-1. 국가별 외국인 전임교원 현황_학교별자료
# 2022년 _대학_6-가-1. 전체 교원 대비 전임교원 현황_학교별자료 #2021년_2024이 각각 다름
# 2022년 _대학_12-아. 산업체 경력 전임교원 현황_학교별자료
# 2022년 _대학_12-거. 기술지주회사 운영 현황_학교별자료
# 2023년 _대학_12-차. 공동활용 연구장비 운영 현황_학교별자료
# 2023년 _대학_13-나. 대학 부설 연구소 현황_학교별자료
# 2023년 _대학_12-거. 기술지주회사 운영 현황_학교별자료
# 2023년 _대학_6-나-(2). 전임교원 1인당 학생 수 및 전임교원 확보율(사이버대학)(2017~)_학교별자료

#4. 대학재정, 교육
# 2023년 _대학_14-차. 강사 강의료_학교별자료
# 2024년 _대학_8-마-2. 자금계산서(결산)_학교별자료
# 2024년 _대학_8-마-4. 대차대조표_학교별자료
# 2024년 _대학_8-마-3. 운영(손익) 계산서_학교별자료

##################################################################################################
# DB 및 작업 폴더 설정
##################################################################################################

# %load_ext autoreload
# %autoreload 2

dbname = 'swu_external'

#conn, curs = db_setting('133.186.247.101', 'root', 'coss!@#$', dbname = 'cosshubtest') #coss test
conn, curs = db_setting('13.124.114.176', 'root', 's!W@u#12', dbname = dbname)

sheet_name = 'Sheet1'
table = 'university'

# template_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/EXCEL/TEMPLATE/1.학생-Template'
# excel_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/EXCEL/1.학생/1.학생 - 2024년'

# template_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/EXCEL/TEMPLATE/2.교육여건-Template'
# excel_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/EXCEL/2.교육여건/2.교육여건 - 2024년'

# template_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/EXCEL/TEMPLATE/3.교육연구성과-Template'
# excel_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/EXCEL/3.교육연구성과/3.교육연구성과 - 2024년'

template_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/EXCEL/TEMPLATE/4.대학재정,교육-Template'
excel_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/EXCEL/4.대학재정,교육/4.대학재정교육 - 2024년'

# template_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/EXCEL/TEMPLATE/5.대학운영-Template'
# excel_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/EXCEL/5.대학운영/5.대학운영 - 2024년'

#template_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/12.학술정보통계시스템/EXCEL/TEMPLATE'
#excel_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/12.학술정보통계시스템/EXCEL'


template_file_list = read_filename(template_dir)
excel_file_list = read_filename(excel_dir)

_dbg_('template_file_list', template_file_list)

# university 테이블 생성
# create_university(dbname, curs, conn, reset_mode = False)




##################################################################################################
## 엑셀데이터를 DB에 추가하는 메인 작업 프로세스
##################################################################################################

excel_worked = [
    #{'doc_title' : '작업문서명', 'db_table' : '테이블명', 'excel_file' : '엑셀파일명'}, 
    #{'doc_title' : '작업문서명', 'db_table' : '테이블명', 'excel_file' : '엑셀파일명'}, 
]



for excel in excel_file_list:
    for template in template_file_list:
        if excel[4:] == template[4:]:
            print("\n\n")
            print("#########################################################################################################")
            print("##### 작업 엑셀 파일명 : ", excel)
            print("#########################################################################################################")

            template_file = '{}/{}'.format(template_dir, template)
            excel_file = '{}/{}'.format(excel_dir, excel)
            
            df_excel, doc_title, df_template_shape0 = compare_schema(template_file, excel_file, sheet_name) #get_table_name(doc_title) 에서 사용
            
            table_name = get_table_name(doc_title)
            print("#########################################################################################################")
            print("##### 작업 테이블명 : ", table_name, excel)
            print("#########################################################################################################")              
            
            df_excel, find_year = remove_empty_row1(df_excel, df_template_shape0)

            #df_excel = remove_empty_row2(df_excel)

            df_excel = remove_univ_col(df_excel)
            #df_excel = fill_year_col(df_excel, find_year, col_name='기준연도')   # 2023상반기... 2023하반기와 같이 토글되는 경우들도 있음....
            df_excel = fill_merge_cell(df_excel) #학교명, 정원외 등 병합된 셀 통합
            
            # _dbg_(df_excel.head())
            # sys.exit(0)
            
            df_excel = strip_univ_name(df_excel, col_name='학교명') #대학명에서 좌우공백 제거
            df_excel = set_univ_code(curs, conn, df_excel, dbname, template_file, excel_file, sheet_name) #데이터프레임의 학교명을 코드명으로 변경, DB학교 공백처리...
            

            
            # 이제 데이터를 DB에 입력하면 됨
            # table_name = get_table_name(doc_title)
            _dbg_('{}테이블에 {}데이터 입력 시작 : {}'.format(table_name, sch.table_dic.get(doc_title), doc_title))
            
            ret, mode = create_table(curs, dbname, table = table_name)
            if ret == 'create': #테이블이 없는 경우는 테이블을 생성하고 인서트
                #idx 모드와 무관하게 실행하자
                insert_df(curs, conn, df_excel, dbname, table_name = table_name, mode = ret) #df_excel을 table에 잘 비교해서 업데이트...
            elif ret == 'exist': #테이블이 있는 경우는 기존 테이블에 업데이트
                _dbg_('############################################')
                #get_db_primary_keys(curs, dbname, table_name)
                #get_db_foregn_keys(curs, dbname, table_name)
                update_df(curs, conn, df_excel, dbname, table_name = table_name, mode = ret) #df_excel을 table에 잘 비교해서 업데이트...
            else:
                _dbg_('테이블 생성 오류 : {}'.format(table_name))
                sys.exit()

            print("\n#########################################################################################################")
            print('##### 작업 완료 : 테이블 = {}, 문서명={}, 엑셀파일={}'.format(table_name, doc_title, excel_file))
            print("#########################################################################################################\n")

            dict_item = {'doc_title' : '{}'.format(doc_title), 'db_table' : '{}'.format(table_name), 'excel_file' : '{}'.format(excel_file)}
            excel_worked.append(dict_item)


if len(excel_worked) <= 0:
    print("\n#########################################################################################################")
    print('##### 작업대상 파일이 없음(템플릿 경로 확인 필요) : {}개'.format(len(excel_worked)))
    print("#########################################################################################################\n")    
else:
    print("\n#######################################################################################################################################################################")
    print('##### 전체 작업 성공 : {}개'.format(len(excel_worked)))
    print("#######################################################################################################################################################################\n")

    for n in range(len(excel_worked)):
        print('\n----- ', n+1, '번째 작업파일 정보 ------')
        print('문서명 : ', excel_worked[n]['doc_title'])
        print('테이블 : ', excel_worked[n]['db_table'])
        print('파일명 : ', excel_worked[n]['excel_file'])
            
            


In [ ]:
sys.exit()


In [ ]:
##################################################################################################
##################################################################################################
# 단독 작업
# DB에 입력된 데이터 검증 : 테이블명, 커맨트, 년도, 년도별 입력된 갯수 확인
##################################################################################################
##################################################################################################
table_cnt = get_db_data_check(dbname, conn)
print("\n############################################################################################################################")
print('#### 검증 작업 테이블 수 : ', table_cnt)
print("############################################################################################################################")



# 이하는 별도 작업 처리

##################################################################################################
# university 테이블 생성 및 업데이트, 단독작업 >> 대학테이블 변경되었음 24-03-21
# 01.Codes.xlsx 파일 기준
##################################################################################################

def create_university(dbname, curs, conn, reset_mode = False):
    excel_univ = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/05.설계/01.테이블설계/01.Codes.xlsx'
    sheet_name = '학교코드정보'

    table_name = 'university'

    df_univ = pd.read_excel(excel_univ, sheet_name = sheet_name)

    df_univ = df_univ.fillna('None')

    cols_ko = list(df_univ.columns) #컬럼 시리즈를 리스트로 변경
    cols_en = list(df_univ.iloc[0]) #데이터 시리즈를 리스트로 변경

    for n in range(len(cols_ko)):
        #print(cols_en[n], cols_ko[n])
        df_univ.rename(columns={cols_ko[n]:cols_en[n]}, inplace=True) #특정 컬럼명 변경
    df = df_univ.drop(0) #특정행 삭제


    # 기존 테이블을 삭제 후 인서트하는 경우
    if reset_mode == True:
        sql_truncate = 'truncate table {}.{}'.format(dbname, table_name)
        curs.execute(sql_truncate)
        conn.commit()

    ret, mode = create_table(curs, dbname, table = table_name)
    _dbg_('테이블 생성 정보 : ', ret, mode)

    if ret == 'create' or ret == 'exist': #create, exist
        #idx 모드와 무관하게 실행하자
        #insert_df(curs, conn, df_excel, dbname, table = table_name) #df_excel을 table에 잘 비교해서 업데이트...
            
        # data insert to db
        field_names = ''
        field_data = ''

        for n in range(len(cols_en)):
            if n == 0 :
                field_names = cols_en[n]
                field_data = '%s'
            else:
                field_names = field_names + ', ' + cols_en[n]
                field_data = field_data + ', ' + '%s'
            
            
        # sql_insert = 'insert into cosshubtest.university(univ_cd, univ_name_arlimi, univ_name, univ_name_en, univ_type, establish, status, campus, campus_name, locale, homepage, tel, fax) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
        # print(sql_insert)

        sql_insert = 'insert into {}.{}({}) values({})'.format(dbname, table_name, field_names, field_data)
        #print(sql)
        for idx in range(len(df)):
            curs.execute(sql_insert, tuple(df.values[idx]))
        conn.commit()

        print("\n#########################################################################################################")
        print('##### university 테이블 생성 완료 : 테이블 = {}, 엑셀파일={}'.format(table_name,  excel_univ))
        print("#########################################################################################################\n")

In [ ]:
###
## 테스트 코드
###

print(sch.table_dic['school_operating_state_non_tuition_account'])
print(sch.table_dic['교비_운영계산서_비등록금회계'])
print(sch.table_dic['대학교정보'])

key_to_check = 'school_operating_statement_non_tuition_account'

try:
    value = sch.table_dic[key_to_check]
    print(value)
except KeyError:
    print("딕셔너리 안에 해당 키가 없습니다.")
    
    
for k in sch.table_dic:
    print(k)

In [ ]:
#####################################################################################################################################################################################################################################################
# university 테이블에만 학교코드를 기준으로 기준에 있던 데이터를 엑셀데이터 기준으로 업데이트하는 모듈
#####################################################################################################################################################################################################################################################

### 대학테이블의 변경 사항을 엑셀기준으로 업데이트

#학교코드	 학교명	              대학구분	    대학규모	   지역	    지역(상세)	    설립구분	 대학명(일반명칭)	  캠퍼스	  캠퍼스명	      법인명	        상태	개교일	    폐교일	    학교 홈페이지	전화번호	팩스번호	비고
#univ_cd    univ_name_arlimi	 univ_type	  scale	        locale	 locale_detail	establish	univ_name	        campus	    campus_name	   corporate_name	status	founding	closing	   homepage	      tel	     fax	     remark


'''
-- swu_external.university definition

CREATE TABLE `university` (
  `univ_cd` varchar(10) NOT NULL COMMENT '학교코드',
  `univ_name_arlimi` varchar(100) DEFAULT NULL COMMENT '학교명(알리미기준)',
  `univ_type` varchar(50) DEFAULT NULL COMMENT '대학구분',
  `scale` varchar(10) DEFAULT NULL COMMENT '대학규모',
  `locale` varchar(10) DEFAULT NULL COMMENT '지역',
  `locale_detail` varchar(50) DEFAULT NULL COMMENT '지역(상세)',
  `establish` varchar(10) DEFAULT NULL COMMENT '설립구분',
  `univ_name` varchar(100) DEFAULT NULL COMMENT '학교명(별칭)',
  `univ_name_en` varchar(100) DEFAULT NULL COMMENT '학교명(영문)',
  `campus` varchar(10) DEFAULT NULL COMMENT '캠퍼스',
  `campus_name` varchar(50) DEFAULT NULL COMMENT '캠퍼스명',
  `corporate_name` varchar(50) DEFAULT NULL COMMENT '법인명',
  `status` varchar(10) DEFAULT NULL COMMENT '상태',
  `founding` varchar(20) DEFAULT NULL COMMENT '개교일',
  `closing` varchar(20) DEFAULT NULL COMMENT '폐교일',
  `homepage` varchar(50) DEFAULT NULL COMMENT '학교 홈페이지',
  `tel` varchar(20) DEFAULT NULL COMMENT '전화번호',
  `fax` varchar(20) DEFAULT NULL COMMENT '팩스번호',
  `remark` varchar(200) DEFAULT NULL COMMENT '비고',
  `reg_date` datetime DEFAULT current_timestamp() COMMENT '생성일시',
  `mod_date` datetime DEFAULT NULL COMMENT '수정일시',
  PRIMARY KEY (`univ_cd`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci COMMENT='대학교정보';
'''




In [ ]:
!pip install numpy

In [ ]:


#################################################
# university 테이블, 학교코드기준 데이터 업데이트
#################################################

import numpy as np

def update_table(curs, conn, df_excel, dbname, table, mode = 'update'):
    _dbg_(table, '에 데이터 업데이트 시작')
    cnt_insert = 0
    cnt_update = 0

    #DB 테이블의 헤더정보만 추출    
    col_info, col_name_org, col_name_list = get_db_column_name(curs, dbname, table) #리턴값중 col_info만 사용

    # 테스트코드 데이터 타입만 읽어오기    
    # data_type = get_data_type(col_info, 'year')
    # _dbg_(data_type)
    
    _dbg_('col_info : ', len(col_info), col_info)
    _dbg_('col_name_org : ', len(col_name_org), col_name_org)
    _dbg_('col_name_list : ', len(col_name_list), col_name_list)
    

    # reg_date와 mod_date는 처리에서 제외
    col_info_cnt = len(col_info)
    for i in range(len(col_info)):
        if col_info[i]['COLUMN_NAME'] == 'reg_date' and col_info[i]['DATA_TYPE'] == 'datetime':
            col_info_cnt = col_info_cnt - 1
        elif col_info[i]['COLUMN_NAME'] == 'mod_date' and col_info[i]['DATA_TYPE'] == 'datetime':
            col_info_cnt = col_info_cnt - 1
            
    # print(df_excel.head(15))
    # sys.exit(1)

    col_header = []
    #col_header = ['univ_cd', 'univ_name_arlimi', 'univ_type', 'scale', 'locale', 'locale_detail', 'establish', 'univ_name', 'univ_name_en', 'campus', 'campus_name', 'corporate_name', 'status', 'founding', 'closing', 'homepage', 'tel', 'fax', 'remark']
    #df_excel 데이터프레임에서 읽어온 데이터와 테이블에 있는 데이터를 비교해서 변경된 사항은 업데이트
    for n in range(len(df_excel)): #전처리 완료된 데이터 프레임 전체를 읽어서 DB에 업데이트 또는 인서트

        field_data = list(df_excel.iloc[n]) # 시리즈 -> 리스트
        _dbg_('업데이트할 행 data : [', n, '] ', field_data)
        
        set_data = ''
        select_where = ''
        
        #if np.isnan(field_data[n]):
        #if field_data[0] == '' or field_data[0] == 'nan' or field_data[0] == None or np.isnan(field_data[0]):
        
        if isinstance(field_data[0], str):
            if len(field_data[0]) == 0:
                break
        else:
            if np.isnan(field_data[0]):
                break
        
        if n == 0:
            col_header = field_data
        else:
            for v in range(len(field_data)):
                
                data = ''
                #if field_data[v] != 'univ_cd':
                if v == len(field_data)-1:
                    if isinstance(field_data[v], str):
                        data = col_header[v] + "='" +  field_data[v] + "'"
                    # elif isinstance(field_data[v], int):
                    #     print("It's a string.")
                    else:
                        data = col_header[v] + "='" +  str(field_data[v]) + "'"
                elif v != 0:
                    if isinstance(field_data[v], str):
                        data = col_header[v] + "='" +  field_data[v] + "', "    
                    # elif isinstance(field_data[v], int):
                    #     print("It's a string.")
                    else:
                        data = col_header[v] + "='" +  str(field_data[v]) + "', "    
                    
                set_data = set_data + data

        
        #UPDATE swu_external.university SET univ_name_arlimi='경주대학교' WHERE univ_cd='1100033'
        if field_data[0] != 'univ_cd':
            sql = "UPDATE {}.{} SET {} WHERE univ_cd = '{}'".format(dbname, table, set_data, field_data[0])
            _dbg_('update : ', sql)
            
            if sql != '':
                curs.execute(sql)
            conn.commit()
            cnt_update = cnt_update+1
    
    print(table, '테이블 : ', cnt_insert, '개의 신규 인서트     ', cnt_update, '개의 업데이트')        




##################################################################################################
# university 테이블, 단독작업 >> 대학테이블 변경되었음 24-03-21
##################################################################################################

def update_university(dbname, curs, conn, reset_mode = False):
    excel_univ = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/11.학과분류체계/학과정보 통합 작업.xlsx'
    sheet_name = '대학코드 (통합완료)'

    table_name = 'university' #DB 테이블명

    df_univ = pd.read_excel(excel_univ, sheet_name = sheet_name)

    #df_univ = df_univ.fillna('None')

    cols_title1 = list(df_univ.columns) #컬럼 시리즈를 리스트로 변경
    cols_title2 = list(df_univ.iloc[0]) #데이터 시리즈를 리스트로 변경

    for n in range(len(cols_title1)):
        #print(cols_en[n], cols_ko[n])
        df_univ.rename(columns={cols_title1[n]:cols_title2[n]}, inplace=True) #특정 컬럼명 변경
    #df = df_univ.drop(0) #특정행 삭제

    #ret, mode = create_table(curs, dbname, table = table_name)
    sql = sch.table_dic[table_name]
    chk_tbl = chk_table(curs, dbname, table_name) #1이상이면 테이블이 존재함
        
    if chk_tbl != 1:
        _dbg_(table_name, "테이블이 없어서 종료")
        sys.exit(0)
        
    _dbg_(cols_title1)
    _dbg_(cols_title2)

    # if ret == 'create' or ret == 'exist': #create, exist
    #     #idx 모드와 무관하게 실행하자
    #     #insert_df(curs, conn, df_excel, dbname, table = table_name) #df_excel을 table에 잘 비교해서 업데이트...

    # data insert to db
    field_names = ''
    field_data = ''
    for n in range(len(cols_title2)):
        if n == 0 :
            field_names = cols_title2[n]
            field_data = '%s'
        else:
            field_names = field_names + ', ' + cols_title2[n]
            field_data = field_data + ', ' + '%s'
        
        _dbg_(field_names, field_data)
        
    update_table(curs, conn, df_univ, dbname, table_name, mode = 'update')
        
    # sql_insert = 'insert into cosshubtest.university(univ_cd, univ_name_arlimi, univ_type, scale, locale, locale_detail, establish, univ_name, univ_name_en, campus, campus_name, corporate_name, status, founding, closing, homepage, tel, fax, remark) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
    # print(sql_insert)

    #sql_insert = 'insert into {}.{}({}) values({})'.format(dbname, table_name, field_names, field_data)
        #print(sql)
    #     for idx in range(len(df)):
    #         curs.execute(sql_insert, tuple(df.values[idx]))
    #     conn.commit()

    #     print("\n#########################################################################################################")
    #     print('##### university 테이블 업데이트 완료 : 테이블 = {}, 엑셀파일={}'.format(table_name,  excel_univ))
    #     print("#########################################################################################################\n")
    
    
    
    

In [ ]:
##########################################################################################################
##########################################################################################################
# 단독작업, 대학 테이블을 엑셀기준으로 업데이트
##########################################################################################################
##########################################################################################################
# update_university(dbname, curs, conn, reset_mode = False)

# 캠퍼스 통합 별도 작업 처리

In [ ]:
#####################################################################################################################################################################################################################################################
#####################################################################################################################################################################################################################################################
# 학교 캠퍼스를 하나의 학교로 데이터를 통합하는 모듈 (기존데이터는 그대로 두고 신규 09XXXXXXX 인 코드를 만들어서 09번 코드학교에 합계를 입력)
#####################################################################################################################################################################################################################################################
#####################################################################################################################################################################################################################################################

'''
통합된 신규 학교코드	학교명		 1캠코드	 2캠코드	  3캠코드	   4캠코드	    5캠코드
univ_cd	univ_name_arlimi						
0900001	국립강릉원주대학교	         0000001    0000002			
0900003	강원대학교		            0000003     0000004			
0900023	전남대학교		            0000023     0000024
0900046	가톨릭대학교                0000046     0000048     0000049		
0900054	건양대학교                  0000054	    0000055			
0900056	경기대학교                  0000056	    0000058			
0900060	경동대학교                  0000060	    0002573	    0002574	    0002744	
0900082	단국대학교                  0000082	    0002726			
0900109	명지대학교                  0000109	    0000111			
0900117	상명대학교                  0000117	    0002959			
0900147	안양대학교                  0000147	    0000148			
0900161	을지대학교                  0000161	    0000162	    0002911
0900164	인제대학교                  0000164	    0000166			
0900167	인천가톨릭대학교            0000167	    0000168			
0900175	중앙대학교                  0000175	    0000174			
0900212	홍익대학교                  0000212	    0002949			
0900218	예원예술대학교              0000218	    0000219			
0900236	영산대학교                  0000236	    0000237			
0900256	경인교육대학교              0000256	    0000257			
0902712	신한대학교                  0002712	    0002800			
0902748	가야대학교(김해)            0902748	    0002749			
'''

#0열 : 이름참고용, 1열:통합되는 기준코드, 2열이상 : 통합대상 코드
#combination_rule에 2열과 3열이 존재한다면 1열기준으로 통합을 수행해야 함
combination_rule = [
    ['국립강릉원주대학교', '0900001', '0000001', '0000002'], 
    ['강원대학교', '0900003', '0000003', '0000004'], 
    ['전남대학교', '0900023', '0000023', '0000024'], 
    ['가톨릭대학교', '0900046', '0000046', '0000048', '0000049'], 
    ['건양대학교', '0900054', '0000054', '0000055'], 
    ['경기대학교', '0900056', '0000056', '0000058'], 
    ['경동대학교', '0900060', '0000060', '0002573', '0002744'], 
    ['단국대학교', '0900082', '0000082', '0002726'], 
    ['명지대학교', '0900109', '0000109', '0000111'], 
    ['상명대학교', '0900117', '0000117', '0002959'], 
    ['안양대학교', '0900147', '0000147', '0000148'], 
    ['을지대학교', '0900161', '0000161', '0000162', '0002911'], 
    ['인제대학교', '0900164', '0000164', '0000166'], 
    ['인천가톨릭대학교', '0900167', '0000167', '0000168'], 
    ['중앙대학교', '0900175', '0000175', '0000174'], 
    ['홍익대학교', '0900212', '0000212', '0002949'], 
    ['예원예술대학교', '0900218', '0000218', '0000219'], 
    ['영산대학교', '0900236', '0000236', '0000237'], 
    ['경인교육대학교', '0900256', '0000256', '0000257'], 
    ['신한대학교', '0902712', '0002712', '0002800'], 
    ['가야대학교(김해)', '0902748', '0902748', '0002749']
]

# 이전에 임시로 부여했던 코드들에 대해서 알아낸 코드로 대체
# 0열이 old 학교코드이고 1열이 new 학교코드
old_and_new_univ_cd = [
    ['1000008', '0000035'], 
    ['1000007', '0000065'], 
    ['1000002', '0000079'], 
    ['1000022', '0000079'], 
    ['1000020', '0000101'], 
    ['1000003', '0000145'], 
    ['1000011', '0000166'], 
    ['1000001', '0000215'], 
    ['1000006', '0000233'], 
    ['1000021', '0000233'], 
    ['1000009', '1100021'], #'0000240'], 
    ['1000010', '0000240'], 
    ['1000016', '1100240'], #'0000240'], 
    ['1000019', '0000242'], 
    ['1000017', '1100244'], #'0000244'], 
    ['1000018', '1100245'], #'0000244'], 
    ['1000025', '0000245'], 
    ['1000026', '0000246'], 
    ['1000004', '0000247'], 
    ['1000012', '0000272'], 
    ['1000013', '0000272'], 
    ['1000023', '0000272'], 
    ['1000005', '0003139'], 
    ['1000024', '0003183'], 
    ['1000027', '1100025'], 
    ['1000014', '1100029'], 
    ['1000015', '1100029'],
    ['0000218-2', '0000219'], 
    ['0002712-2', '0002800'], 
    ['0000162-3', '0002911'], 
    ['0000147-2', '0000148']
]

# 기존테이블 중에 통합 작업을 하지 않는 테이블 리스트 > 이테이블들에 대한 별도 작업필요
except_table = [
    'group_info', 
    'group_univ', 
    'employment_type', 
    'swu_fund', 
    'university', 
    'university_old', 
    'swu_fund', 
    'vw_education_expenses', 
    'vw_professor_acquisition_rate_per_student_ranking', 
    'vw_field_training',
    
    #idx 있음 > 나중에 고민
    'admission_result_type', 
    'contract_department',
    'ondemand_curriculum', 

    #키가 4개 이상
    'student_loan_status', #4
    'dormitory_status', 
    'instructor_fee'
]

# 오류로 처리가 안된 테이블 리스트
error_table = [
    ##### 오류로 못한 키 두개짜리 처리 완료 (_variance의 _가 없어서 발생한 문제임)
    # 'school_operating_state_non_tuition_account',
    # 'income_statement',
    # 'school_operating_state_tuition_account',
    # 'school_operating_state_tot',
    # 'corporation_operating_statement'
    
    ##### 오류로 못한 키 3개짜리 (키의 순서가 year, term, univ_cd 와 같이 들어가서 발생한 문제임)
    #'student_recruitment_rate',
    # 'course_by_enrollment_size',
    
    
    
    # ## 'dormitory_payment_system', 오류
    # 'foreign_professor_department_status', #3번째 키가 공학계열, 자연계열 등등 6개나 되기때문에 복잡해서...
    # 'foreign_professor_status', #3번째 키가 미국, 중국, 일본... 등등 6개나 되기때문에 복잡해서...
]

# 키가 3개 이상인 테이블... semester 등 별도 작업 필요
# 키가 3개인 테이블 AND 키가 3개의 필드를 포함하는 경우의 통합 작업
##### 는 오류로 멈춤
key3_table = [
    # semester 컬럼을 갖는 테이블
    'course_by_enrollment_size',
    'lecture_responsibility_ratio', 
    'liberal_arts_grades', 
    'major_grades', 
    'teaching_profession_grades', 
    
    # term 컬럼을 갖는 테이블
    'student_recruitment_rate', 
    
    # type 컬럼을 갖는 테이블
    'tuition_reduction_status', 
    
    # dormitory_type 컬럼을 갖는 테이블
    'dormitory_payment_system',
    'dormitory_result_private_univ_capital',
    
    # department 컬럼을 갖는 테이블
    'foreign_professor_department_status', 
    
    # country 컬럼을 갖는 테이블
    'foreign_professor_status',
    
    # account_type 컬럼을 갖는 테이블
    'reserve_fund_status', 
    
    # equipment_name 컬럼을 갖는 테이블
    'shared_research_equipment', 
    
    # laboratory_name 컬럼을 갖는 테이블
    'univ_research_institute' 
]

# 키가 3개인 테이블의 컬럼명
key3_field = [
    'semester', #'course_by_enrollment_size',     'lecture_responsibility_ratio',     'liberal_arts_grades',     'major_grades',     'teaching_profession_grades', 
    'term', #'student_recruitment_rate', 
    'type', #'tuition_reduction_status', 
    'dormitory_type',      #'dormitory_payment_system', 'dormitory_result_private_univ_capital',
    'department',     #'foreign_professor_department_status', 
    'country',     #'foreign_professor_status',
    'account_type',     #'reserve_fund_status', 
    'equipment_name',     #'shared_research_equipment', 
    'laboratory_name'     #'univ_research_institute' 
]



# 필드명중 단순 합산해서는 안되는 필드명 지정, 평균으로 해야할 수도...
except_field = [
    'idx', 
    'year', 
    'univ_cd', 
    'semester',
    'dormitory_type', #dormitory_payment_system, dormitory_result_private_univ_capital
    'dormitory_name', #dormitory_status
    'reg_date', 
    'mod_date',
]

# 필드명중 단순 합산해서는 안되는 필드명 지정, 평균으로 해야하는 필드
except_field_avg = [
    'none', 
    
    #중도탈락율로 테스트함
    'dropout_case_rate', #중도탈락 학생 비율(사유별 중도탈락_총계/재적학생 총계)
    'fresh_dropout_case_rate' #신입생_중도탈락 학생 비율(신입생_사유별 중도탈락 학생_총계/재적학생 총계)
]

# for row in combination_rule:
#     for element in row:
#         print(element, end=" | ")
#     print()
    
    
# for row in range(len(combination_rule)):
#     for col in range(len(combination_rule[row])):
#         print(combination_rule[row][col], end=" | ")
#     print()
    
#################################################
# 통합할 코드들 읽기
#################################################
def get_target_univcode(combination_rule):
    print()
    
#################################################
# DB의 전체 테이블 리스트 읽어오기
#################################################
def get_db_table_list(curs, dbname):
    #sql = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';
    sql = "SELECT table_name FROM information_schema.tables WHERE table_schema = '{}'".format(dbname)
    rs_cnt = curs.execute(sql)
    rs_dic_list = curs.fetchall() #전체 테이블 목록
    
    table_dic_list = []
    #_dbg_(rs_cnt)
    #_dbg_(table_dic_list)
    
    for n in range(len(rs_dic_list)):
        #_dbg_(rs_dic_list[n]['table_name'])
        table_dic_list.append(rs_dic_list[n]['table_name'])    
    
    return table_dic_list

#################################################
# 테이블의 프라이머리 키 리스트 읽어오기
#################################################
def get_db_table_keys(curs, dbname, table_name):
    #sql = "SELECT pg_constraint.conname AS pk_name FROM pg_constraint JOIN pg_class ON pg_constraint.conrelid = pg_class.oid WHERE pg_class.relname = 'your_table_name' AND pg_constraint.contype = 'p'".format(table_name)
    sql = "SHOW KEYS FROM {}.{} WHERE Key_name = 'PRIMARY'".format(dbname, table_name)
    rs_cnt = curs.execute(sql)
    rs_dic_list = curs.fetchall() #결과 목록
    
    pk_list = []
    
    #_dbg_(rs_cnt)
    #_dbg_(table_dic_list)
    for n in range(len(rs_dic_list)):
        #_dbg_(rs_dic_list[n]['Column_name'])
        pk_list.append(rs_dic_list[n]['Column_name'])
    
    return pk_list

#################################################
# 테이블의 컬럼영에 대한 데이터타입 찾기
#################################################
def get_table_col_type(curs, dbname, table_name, col_name):
    sql = "SELECT DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = '{}' AND TABLE_NAME = '{}' AND COLUMN_NAME = '{}'".format(dbname, table_name, col_name)
    #_dbg_(sql)
    rs_cnt = curs.execute(sql)
    rs_dic_list = curs.fetchall() #결과 목록

    return rs_dic_list


#################################################
# 테이블의 year key 에 해당하는 전체 컬럼값을 읽어서 중복값을 제거
#################################################
def get_db_year_col(curs, dbname, table_name):
    #key_list = get_db_table_keys(curs, dbname, table_name)
    sql = "SELECT {} FROM {}.{}".format('year', dbname, table_name)
    rs_cnt = curs.execute(sql)
    rs_dic_list = curs.fetchall() #결과 목록
    
    year_list = []
    for n in range(len(rs_dic_list)):
        year_list.append(rs_dic_list[n]['year'])
        
    year_list = list(set(year_list)) #중복값 제거
    #_dbg_(year_list)
    
    return year_list






#################################################
# 테이블에 통합대상 캠퍼스들이 있는지 확인
#################################################
def chk_table_and_rule(curs, dbname, table_name, combination_rule):
    key_list = get_db_table_keys(curs, dbname, table_name)
    
    #['year', 'univ_cd'] 2개의 키값을 갖는 데이터들만 처리
    if len(key_list) == 2: 
        #sql = "UPDATE {}.{} SET {} WHERE univ_cd = '{}'".format(dbname, table, set_data, field_data[0])
        sql = "SELECT * FROM {}.{} WHERE {} = {} and {} = '{}'".format(dbname, table_name, key_list[0], 2023, key_list[1], combination_rule[0][2])
        _dbg_('쿼리 검사해봐야함')
        sys.exit()
        rs_cnt = curs.execute(sql)
        rs_dic_list = curs.fetchall() #결과 목록
        _dbg_(rs_cnt, rs_dic_list)
        
#################################################
# 기존 통합된 09xxx 레코드가 있다면 삭제 후 다시 작업
#################################################
#def delete_old_09xx(curs, conn, dbname, table_name, year, uname_09xx):
def delete_old_09xx(curs, conn, dbname, table_name, year_list, combination_rule):
    #sql_del = "DELETE FROM {}.{} WHERE  year = {} and univ_cd = '{}'".format(dbname, table_name, year, uname_09xx)
    
    del_cnt = 0
    #year_list별로 combination_rule[2]이상 코드에 대한 확인 작업 수행
    for year in year_list:
        #combination_rule의 각 통합 대상 대학별로...
        for unames in range(len(combination_rule)): 
            sql_del = "DELETE FROM {}.{} WHERE  year = {} and univ_cd = '{}'".format(dbname, table_name, year, combination_rule[unames][1])
            _dbg_('sql_del', sql_del)
            cnt = curs.execute(sql_del)
            del_cnt = del_cnt + cnt
    ret = conn.commit()
    _dbg_(table_name, '테이블에',  del_cnt, '개의 기존 레코드 삭제 execute : ', ret) 

#################################################
# combination_rule의 각 통합 대상 대학의 분교들을 찾음
#################################################
def select_target_branch(curs, dbname, table_name, rs_branch_dict_list, rs_branch_dict_cnt, year, combination_rule, unames, ubranch, key3_k, key3_v, key_mode = 2):
    rs_dic_list = []
    sql = ''
    if key_mode == 2: #year + univ_cd
        sql = "SELECT * FROM {}.{} WHERE year = {} and univ_cd = '{}'".format(dbname, table_name, year, combination_rule[unames][ubranch + 2]) #col은 2번째 컬럼 이상에서만
    elif key_mode == 3: #year + univ_cd + 3key
        sql = "SELECT * FROM {}.{} WHERE year = {} and univ_cd = '{}' and {} = '{}'".format(dbname, table_name, year, combination_rule[unames][ubranch + 2], key3_k, key3_v) #col은 2번째 컬럼 이상에서만
    rs_cnt = curs.execute(sql)
    rs_dic_list = curs.fetchall() #결과 목록 (딕셔너리 리스트)
    #_dbg_('\nrs_dic_list 1', rs_dic_list) #정상적으로 rs_dic_list 리스트에 들어옴
    
    #1개 이상의 결과가 나올수도 있다. 키가 3개 있는 경우....
    if rs_cnt >= 1: 
        #_dbg_(combination_rule[unames][0], combination_rule[unames][ubranch + 1], table_name, year, combination_rule[unames][ubranch + 2])    
        _dbg_('작업대상추가', rs_dic_list)
        #_dbg_('rs_dic_list', type(rs_dic_list), rs_dic_list) #리스트
        #_dbg_('rs_dic_list[0]', type(rs_dic_list[0]), rs_dic_list[0]) #딕셔너리
        
        #rs_combi_dict_list[rs_combi_dict_cnt] = rs_dic_list[0].copy()
        #rs_combi_dict_list[rs_combi_dict_cnt] = rs_dic_list
        
        #_dbg_('rs_dic_list 2', rs_dic_list) #정상적으로 rs_dic_list 리스트에 들어옴
        rs_branch_dict_list.append(rs_dic_list)
        #_dbg_('rs_branch_dict_list에 추가', len(rs_branch_dict_list), rs_dic_list) #리스트
        #_dbg_('rs_branch_dict_list', type(rs_branch_dict_list), rs_branch_dict_list) #리스트
        #_dbg_('rs_branch_dict_list에 추가', rs_branch_dict_list[0]) #리스트
        
        rs_branch_dict_cnt += 1
    
    return rs_branch_dict_cnt, rs_branch_dict_list

#################################################
#SQL Select로 검색된 분교 결과(rs_branch_dict_list)에 대해서 09xx 통합코드로 더하기 수행
#################################################
def combi_process(rs_branch_dict_list, unames, combination_rule):
    tmp_val_dict = {} #더해진_딕셔너리
    tmp_val_dict_cnt = 0
    for univ_list in rs_branch_dict_list:
        #_dbg_('univ_list', type(univ_list), len(univ_list), univ_list)
        #_dbg_('univ_list[0]', type(univ_list[0]), len(univ_list[0]), univ_list[0])
        key_list = list(univ_list[0].keys()) #키 값만 추출해서 리스트로 저장
        val_list = list(univ_list[0].values()) #키 값만 추출해서 리스트로 저장
        #_dbg_('key_list', key_list)
        #_dbg_('val_list', val_list)
        
        #분교 리스트내의 각 딕셔너리들에 대해서 더하기
        #최초는 일단 기존 딕셔너리 값 복사
        #두번째 분교부터는 첫번째 값들과 더하기
        if tmp_val_dict_cnt == 0:
            for key in univ_list[0]:
                tmp_val_dict[key] = univ_list[0][key]
            tmp_val_dict_cnt += 1
            
            # combi_dict['univ_cd'] 이거는 combination_rule[]에서 찾은 후 첫열의 통합코드 값으로 지정을 해줘야 함
            # tmp_val_dict['univ_cd'] = '0' + str(int(combination_rule[unames][ubranch + 1]) + 900000) #이렇게해도 되지만... 아래가 더 빠를듯
            tmp_val_dict['univ_cd'] = combination_rule[unames][1]
        else:
            for key in tmp_val_dict:
                if key not in except_field:
                    tmp_val_dict[key] += univ_list[0][key]
                    
                    
        #분교 리스트내의 각 딕셔너리들에 대해서 평균값
        #최초는 일단 기존 딕셔너리 값 복사
        #두번째 분교부터는 첫번째 값들과 평균
        if tmp_val_dict_cnt == 0:
            for key in univ_list[0]:
                tmp_val_dict[key] = univ_list[0][key]
            tmp_val_dict_cnt += 1
            
            # combi_dict['univ_cd'] 이거는 combination_rule[]에서 찾은 후 첫열의 통합코드 값으로 지정을 해줘야 함
            # tmp_val_dict['univ_cd'] = '0' + str(int(combination_rule[unames][ubranch + 1]) + 900000) #이렇게해도 되지만... 아래가 더 빠를듯
            tmp_val_dict['univ_cd'] = combination_rule[unames][1]
        else:
            for key in tmp_val_dict:
                if key not in except_field:
                    tmp_val_dict[key] += univ_list[0][key]          
                    
                    
                    
    #최종결과는 tmp_val_dict 딕셔너리에 저장되어있음
    #이제 이 tmp_val_dict 딕셔너리의 값들을 테이블에 인서트 하면 됌
    #_dbg_('final tmp_val_dict', tmp_val_dict)
    return tmp_val_dict




#################################################
# 통합된 신규 09xxx 레코드 인서트
# univ_combination_code와 univ_combination_3code 에서 공통적으로 사용
#################################################

기존의 데이터는 유지하고 신규년도에 들어온 데이터들만 추가
신규년도에 들어온 데이터들만 기준으로 연산 수행하는 모듈 필요

통합09코드.sql과 09작업.sql 기준으로 편집하는 방안 필요

정수는 합산하고 rate가 포함되어있고 실수인 컬럼들은 평균 -> 실제 데이터를 확인하며 검토 필요

def insert_new_09xx(curs, conn, dbname, table_name, tmp_val_dict):
    keys = ''
    values = ''
    first_cnt = 0
    for k, v in tmp_val_dict.items(): #테이블의 데이터 타입을 보고 지정 and 년도 등을 제외
        if isinstance(v, (int, float)):
            if first_cnt == 0:
                keys = str(k)
                values = str(v)
            else:
                keys = str(keys) + ', ' + str(k)
                values =  str(values) + ', ' +  str(v)
        elif  isinstance(v, (str)):
            if first_cnt == 0:
                keys = str(k)
                values = "'"+ str(v) +"'"
            else:
                keys = str(keys) + ',' + str(k)
                values = str(values) + ", '" + str(v) + "'"                            
        first_cnt += 1  
    sql = 'insert into {}.{}({}) values({})'.format(dbname, table_name, keys, values)
    #sql = 'update {}.{} set {}={} where year={} and univ_cd={}'.format(dbname, table_name, keys, values, year, combi_dict['univ_cd'])
    _dbg_('통합입력', sql)
    
    cnt = curs.execute(sql)
    ret = conn.commit()
    _dbg_('sql execute 결과 ', cnt, ret)
                        
#################################################
# table_dic_list[n] 테이블에서 year + univ_code(combination_rule[2] 이상)가 존재하는지를 확인
# univ_combination_3code가 별도로 존재함
#################################################
def univ_combination_code(dbname, table_name, year_list, combination_rule):
    
    #combination_rule에 2열과 3열이 존재한다면 1열기준으로 통합을 수행해야 함
   
    #year_list별로 combination_rule[2]이상 코드에 대한 확인 작업 수행
    for year in year_list:
        
        #combination_rule의 각 통합 대상 대학별로...
        for unames in range(len(combination_rule)):
            rs_branch_dict_list = [] #sql select 결과를 저장하는 리스트        
            rs_branch_dict_cnt = 0
                    
            #combination_rule의 각 통합 대상 대학의 분교들을 찾음
            for ubranch in range(len(combination_rule[unames]) - 2): #한글명과 통합대상 09XXXXX 코드는 제외하고..
                rs_branch_dict_cnt, rs_branch_dict_list = select_target_branch(curs, dbname, table_name, rs_branch_dict_list, rs_branch_dict_cnt, year, combination_rule, unames, ubranch, key3_k=0, key3_v=0, key_mode = 2)


            #combination_rule기준 unames에 2열과 3열 이상의 ubranch가 존재한다면, 1열의 09XXXXX 기준으로 통합 작업을 수행함
            #rs_combi_dict_cnt가 1개라면 분교가 1개만 검색되었다는 의미이며, 이 경우는 09XXXXX 로 통합할까?
            #일단은 분교가 2개 이상인 경우만 작업하자!rs_branch_dict_cnt >= 2
            #분교가 1개만 있더라도 통합대상 대학에 포함시키자!!!rs_branch_dict_cnt >= 1
            if rs_branch_dict_cnt >= 1: 

                #SQL Select로 검색된 분교 결과(rs_branch_dict_list)에 대해서 09xx 통합코드로 더하기 수행
                tmp_val_dict = combi_process(rs_branch_dict_list, unames, combination_rule)
                
                #통합된 신규 09xxx 레코드 인서트
                insert_new_09xx(curs, conn, dbname, table_name, tmp_val_dict)
        
        #sys.exit(0) 

#################################################
# table_name에서 year, univ_code 이외에 3번째 키에 해당하는 키와 값을 찾음
#################################################
def find_key3_k(dbname, table_name, exclude=True):
    #sql_col = 'SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = "{}"'.format(table) #키와 데이터 타입들 추출
    sql = 'SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = "{}" AND TABLE_NAME = "{}" AND COLUMN_KEY = "PRI"'.format(dbname, table_name) #키값만 추출

    rs_cnt = curs.execute(sql)
    #conn.commit()
    col_key = curs.fetchall() #컬럼명과 데이터타입 리스트 안에 딕셔너리로...
    # _dbg_('col_key', type(col_key), len(col_key), col_key)
    # col_key <class 'list'> 3 [{'COLUMN_NAME': 'year'}, {'COLUMN_NAME': 'univ_cd'}, {'COLUMN_NAME': 'equipment_name'}] 
    
    
    key_cnt = 0
    key_list = []
    key_val = []
    for key in col_key:
        #_dbg_('k', type(key), len(key), key) #k <class 'dict'> 1 {'COLUMN_NAME': 'year'}
        key_list.append(key['COLUMN_NAME'])
    
    _dbg_('key_val', key_list)
    
    exclude_value = []
    
    #기본 'year', 'univ_cd'는 제외
    if exclude == True:
        exclude_value = ['year', 'univ_cd']
        key_cnt = len(key_list) - len(exclude_value)
        
        # 특정 값을 제외한 나머지 값이 들어간 인덱스를 구하는 리스트 컴프리헨션
        #indices = [index for index, value in enumerate(key_val) if value != exclude_value]
        key_idx = [index for index, value in enumerate(key_list) if value not in exclude_value]
        
        #세개의 키만 대상으로 작업을 하기 때문에 indice_list[0]에만 값이 있어야 한다.
        key_val.append(key_list[key_idx[0]])
    else:
        key_cnt = len(key_list)
        key_val = key_list
    
    return key_cnt, key_val


#################################################
# table_name에서 모든 키에 해당하는 키와 값을 찾음
#################################################
def find_key3_cnt(dbname, table_name):
    #sql_col = 'SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = "{}"'.format(table) #키와 데이터 타입들 추출
    sql = 'SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = "{}" AND TABLE_NAME = "{}" AND COLUMN_KEY = "PRI"'.format(dbname, table_name) #키값만 추출

    rs_cnt = curs.execute(sql)
    #conn.commit()
    col_key = curs.fetchall() #컬럼명과 데이터타입 리스트 안에 딕셔너리로...
    # _dbg_('col_key', type(col_key), len(col_key), col_key)
    # col_key <class 'list'> 3 [{'COLUMN_NAME': 'year'}, {'COLUMN_NAME': 'univ_cd'}, {'COLUMN_NAME': 'equipment_name'}] 
    
    key_list = []
    for key in col_key:
        #_dbg_('k', type(key), len(key), key) #k <class 'dict'> 1 {'COLUMN_NAME': 'year'}
        key_list.append(key['COLUMN_NAME'])
    
    key_cnt = len(key_list)
    
    return key_cnt, key_list
    
#################################################
# table_name에서 year, univ_code 이외에 3번째 키에 해당하는 키와 값을 찾음
#################################################
def find_key3_v(dbname, table_name, year, univ_cd, key3_k):
    #_dbg_('year, univ_cd, key3_k', year, univ_cd, key3_k)
    
    sql = "SELECT {} FROM {}.{} WHERE year = {} and univ_cd = '{}'".format(key3_k, dbname, table_name, year, univ_cd) #col은 2번째 컬럼 이상에서만
    rs_cnt = curs.execute(sql)
    #conn.commit()
    col_val = curs.fetchall() #컬럼명과 데이터타입 리스트 안에 딕셔너리로...
    
    #_dbg_(len(col_val), type(col_val), col_val)
    
    return col_val
        
        
# data = [[{'year': 2020, 'univ_cd': '0000001', 'semester': '1학기', 'stud_tot': 235, 'perfect_rating': 4.5, 'stud_Ap': 149, 'rate_Ap': 63.4, 'stud_A0': 44, 'rate_A0': 18.7, 'stud_Bp': 0, 'rate_Bp': 0.0, 'stud_B0': 18, 'rate_B0': 7.7, 'stud_Cp': 12, 'rate_Cp': 5.1, 'stud_C0': 0, 'rate_C0': 0.0, 'stud_Dp': 1, 'rate_Dp': 0.4, 'stud_D0': 5, 'rate_D0': 2.1, 'stud_F': 0, 'rate_F': 0.0, 'reg_date': datetime.datetime(2024, 2, 21, 19, 22, 42), 'mod_date': None}], [{'year': 2020, 'univ_cd': '0000001', 'semester': '2학기', 'stud_tot': 154, 'perfect_rating': 4.5, 'stud_Ap': 66, 'rate_Ap': 42.9, 'stud_A0': 22, 'rate_A0': 14.3, 'stud_Bp': 0, 'rate_Bp': 0.0, 'stud_B0': 59, 'rate_B0': 38.3, 'stud_Cp': 3, 'rate_Cp': 1.9, 'stud_C0': 0, 'rate_C0': 0.0, 'stud_Dp': 0, 'rate_Dp': 0.0, 'stud_D0': 1, 'rate_D0': 0.6, 'stud_F': 0, 'rate_F': 0.0, 'reg_date': datetime.datetime(2024, 2, 21, 19, 22, 42), 'mod_date': None}], [{'year': 2020, 'univ_cd': '0000002', 'semester': '1학기', 'stud_tot': 408, 'perfect_rating': 4.5, 'stud_Ap': 220, 'rate_Ap': 53.9, 'stud_A0': 127, 'rate_A0': 31.1, 'stud_Bp': 0, 'rate_Bp': 0.0, 'stud_B0': 37, 'rate_B0': 9.1, 'stud_Cp': 13, 'rate_Cp': 3.2, 'stud_C0': 0, 'rate_C0': 0.0, 'stud_Dp': 5, 'rate_Dp': 1.2, 'stud_D0': 6, 'rate_D0': 1.5, 'stud_F': 0, 'rate_F': 0.0, 'reg_date': datetime.datetime(2024, 2, 21, 19, 22, 42), 'mod_date': None}], [{'year': 2020, 'univ_cd': '0000002', 'semester': '2학기', 'stud_tot': 267, 'perfect_rating': 4.5, 'stud_Ap': 94, 'rate_Ap': 35.2, 'stud_A0': 50, 'rate_A0': 18.7, 'stud_Bp': 0, 'rate_Bp': 0.0, 'stud_B0': 93, 'rate_B0': 34.8, 'stud_Cp': 17, 'rate_Cp': 6.4, 'stud_C0': 0, 'rate_C0': 0.0, 'stud_Dp': 3, 'rate_Dp': 1.1, 'stud_D0': 5, 'rate_D0': 1.9, 'stud_F': 0, 'rate_F': 0.0, 'reg_date': datetime.datetime(2024, 2, 21, 19, 22, 42), 'mod_date': None}]]

#################################################
# 'semester'을 기준으로 리스트를 분할하는 함수 정의
#################################################
def split_by_key3(data, key3):
    semesters = {}
    for sublist in data:
        for item in sublist:
            semester = item[key3]
            if semester not in semesters:
                semesters[semester] = []
            semesters[semester].append(item)
    return semesters

# # 'semester'을 기준으로 리스트 분할
# semester_data = split_by_semester(data)

# # 결과 출력
# for semester, items in semester_data.items():
#     print(semester, ':', items)


#################################################
#SQL Select로 검색된 분교 결과(rs_branch_dict_list)에 대해서 09xx 통합코드로 더하기 수행
#################################################
def combi_3key_process(curs, conn, dbname, table_name, rs_branch_dict_list, unames, combination_rule):
    tmp_val_dict = {} #더해진_딕셔너리
    tmp_val_dict_cnt = 0
    
    #_dbg_('rs_branch_dict_list', type(rs_branch_dict_list), len(rs_branch_dict_list), rs_branch_dict_list)
    for univ_list in rs_branch_dict_list:
        #_dbg_('univ_list', type(univ_list), len(univ_list), univ_list)
        #_dbg_('univ_list[0]', type(univ_list[0]), len(univ_list[0]), univ_list[0])
        key_list = list(univ_list.keys()) #키 값만 추출해서 리스트로 저장
        val_list = list(univ_list.values()) #키 값만 추출해서 리스트로 저장
        #_dbg_('key_list', key_list)
        #_dbg_('val_list', val_list)
        
        #분교 리스트내의 각 딕셔너리들에 대해서
        #최초는 일단 기존 딕셔너리 값 복사
        #두번째 분교부터는 첫번째 값들과 더하기
        if tmp_val_dict_cnt == 0:
            for key in univ_list:
                tmp_val_dict[key] = univ_list[key]
            tmp_val_dict_cnt += 1
            
            # combi_dict['univ_cd'] 이거는 combination_rule[]에서 찾은 후 첫열의 통합코드 값으로 지정을 해줘야 함
            # tmp_val_dict['univ_cd'] = '0' + str(int(combination_rule[unames][ubranch + 1]) + 900000) #이렇게해도 되지만... 아래가 더 빠를듯
            tmp_val_dict['univ_cd'] = combination_rule[unames][1]
        else:
            for key in tmp_val_dict:
                if key not in except_field:
                    
                    #키 타입이 varchar인 경우는 첫번째 분교 값으로 대체하고, 숫자만 더하기를 수행할것임
                    #db, 테이블, 키값을 통해 해당키의 타입을 찾아내는 함수
                    
                    col_type = get_table_col_type(curs, dbname, table_name, col_name=key)
                    #_dbg_('col_type', col_type)
                    
                    #문자 컬럽의 경우 더하기를 하면 미실시미실시 등과 같이 텍스트 중복이 발생 함
                    if col_type[0]['DATA_TYPE'] != 'varchar':
                        tmp_val_dict[key] += univ_list[key]
                    else:
                        tmp_val_dict[key] = univ_list[key]
    #최종결과는 tmp_val_dict 딕셔너리에 저장되어있음
    #이제 이 tmp_val_dict 딕셔너리의 값들을 테이블에 인서트 하면 됌
    #_dbg_('final tmp_val_dict', tmp_val_dict)
    return tmp_val_dict
        
#################################################
# table_dic_list[n] 테이블에서 year + univ_code(combination_rule[2] 이상)과 추가 키필드가 가 존재하는지를 확인
# univ_combination_code가 별도로 존재함
#################################################
def univ_combination_3code(dbname, table_name, year_list, combination_rule):
    
    #테이블의 3번째 키값 찾기, year+univ_cd는 제외
    key_cnt, key3_k = find_key3_k(dbname, table_name, exclude=True)
    _dbg_('세번째 키수', key_cnt)
    _dbg_('세번째 키값', type(key3_k), len(key3_k), key3_k)
    
    #year_list별로 combination_rule[2]이상 코드에 대한 확인 작업 수행
    for year in year_list:
        
        #combination_rule의 각 통합 대상 대학별로...
        for unames in range(len(combination_rule)):
            rs_branch_dict_list = [] #sql select 결과를 저장하는 리스트        
            rs_branch_dict_cnt = 0

            #combination_rule의 각 통합 대상 대학의 분교들을 찾음
            for ubranch in range(len(combination_rule[unames]) - 2): #한글명과 통합대상 09XXXXX 코드는 제외하고..
                #key3_k, key3_v 값을 찾아서 넣어야함 key3테이블과 값테입2ㅡㄹ
                
                #_dbg_('v1', unames, ubranch)
                #테이블의 3번째 키값의 값찾기
                key3_v_list = find_key3_v(dbname, table_name, year, combination_rule[unames][ubranch+2], key3_k[0]) #key3_k[0]는 키가 3개인 경우만 해당
                #_dbg_('v2', key3_v_list, type(key3_v_list))
                for key3_v in key3_v_list: #1학기 2학기가 또는 미국, 영국, 일본 식으로 키의 값들이 포함됨
                    _dbg_('\n\n\n#### 세번째 키값을 찾아서 통합하는 작업을 수행 ###')
                    _dbg_('세번째 (키, 벨튜타입, 벨류)', key3_k[0], type(key3_v), key3_v[key3_k[0]])
                    rs_branch_dict_cnt, rs_branch_dict_list = select_target_branch(curs, dbname, table_name, rs_branch_dict_list, rs_branch_dict_cnt, year, combination_rule, unames, ubranch, key3_k[0], key3_v[key3_k[0]], key_mode = 3) #key3_k[0]는 키가 3개인 경우만 해당
            _dbg_('>>>> 작업분교의수와리스트', rs_branch_dict_cnt, '\n', rs_branch_dict_list)
                    # 'year': 2021, 'univ_cd': '0000001', 'dormitory_type': '민자', 'credit_card_im
                    # 'year': 2021, 'univ_cd': '0000001', 'dormitory_type': '직영', 'credit_card_im
                    # 'year': 2021, 'univ_cd': '0000002', 'dormitory_type': '민자', 'credit_card_im
                    # 'year': 2021, 'univ_cd': '0000002', 'dormitory_type': '직영', 'credit_card_im


            #combination_rule기준 unames에 2열과 3열 이상의 ubranch가 존재한다면, 1열의 09XXXXX 기준으로 통합 작업을 수행함
            #rs_combi_dict_cnt가 1개라면 분교가 1개만 검색되었다는 의미이며, 이 경우는 09XXXXX 로 통합할까?
            #분교가 1개만 있더라도 통합대상 대학에 포함시키자!!!rs_branch_dict_cnt >= 1
            
            
            if rs_branch_dict_cnt >= 1: 
                
                # for key3_v in key3_v_list: #1학기 2학기가 포함됨
                #     if key3_v == '1학기'
                
                #_dbg_(rs_branch_dict_list)
                
                # 'semester'을 기준으로 리스트 분할
                # '민자': [{'year': 2021, 'univ_cd': '0000001', 'dormitory_type': '민자', 'cre
                #         {'year': 2021, 'univ_cd': '0000002', 'dormitory_type': '민자', 'cre

                # '직영': [{'year': 2021, 'univ_cd': '0000001', 'dormitory_type': '직영', 'cre
                #         {'year': 2021, 'univ_cd': '0000002', 'dormitory_type': '직영', 'cre

                semester_data = split_by_key3(rs_branch_dict_list, key3_k[0])
                _dbg_('\n\n세번째키기준 분류된_전체분교_데이터\n', semester_data)
                # semester_data            
                # '공학계열': [{'year': 2023, 'univ_cd': '0000001', 'department': '공학계열', 
                # '소속학과없음': [{'year': 2023, 'univ_cd': '0000001', 'department': '소속학과없음', 
                # '예ㆍ체능계열': [{'year': 2023, 'univ_cd': '0000001', 'department': '예ㆍ체능계열', 
                # '의학계열': [{'year': 2023, 'univ_cd': '0000001', 'department': '의학계열', 
                # '인문ㆍ사회계열': [{'year': 2023, 'univ_cd': '0000001', 'department': '인문ㆍ사회계열', 
                # '자연과학계열':   
                
                # semester == key3_v_list         

                # year, univ_cd는 일치하지만 semester는 6개도 나올 수 있음 '공학계열', '소속학과없음', '예ㆍ체능계열', '의학계열', '인문ㆍ사회계열', '자연과학계열', ...
                # year, univ_cd + 첫번째 semester 값 부터 루프
                for semester, items in semester_data.items():
                    _dbg_('\n\n3번째 키값으로 분류한 대상 ==================', semester)
                    _dbg_('3번째키 벨류(items)-year, univ_cd, 3키 분류로된 아이템들... 이 아이템들에 대한 더하기를 수행하면 될듯...\n', items)
                    
                    # key3_v_list           
                    # 0:{'department': '공학계열'}
                    # 1:{'department': '소속학과없음'}
                    # 2:{'department': '예ㆍ체능계열'}
                    # 3:{'department': '의학계열'}
                    # 4:{'department': '인문ㆍ사회계열'}
                    # 5:{'department': '자연과학계열'}
                    
                    # items
                    # 0:{'year': 2023, 'univ_cd': '0000001', 'department': '공학계열', 'fulltime_tot_male': ...
                    # 1:{'year': 2023, 'univ_cd': '0000002', 'department': '공학계열', 'fulltime_tot_male': ...
                    
                    
                    # '''
                    # semester_sep = []
                    # for key3_n in range(len(key3_v_list)):
                    #     #_dbg_('key3_n', key3_n, type(items[key3_n]), key3_v_list[key3_n])
                    #     #_dbg_('len(items)', len(items), items, items[key3_n])
                        
                    #     if len(items) > 1:
                    #         #if items is not Null:
                    #         semester_sep.append(items[key3_n])
                    #         #_dbg_('semester_sep > 1에 추가됨', semester_sep)
                    #     elif len(items) == 1:
                    #         semester_sep.append(items[0])
                    #         #_dbg_('semester_sep == 1에 추가됨', semester_sep)
                    # '''
                   
                    
                    #_dbg_('\n\nsemester_sep 추가됨(이값들을 더하기 할것임)', semester_sep)
                    
                    #_dbg_('111111', type(semester_sep), len(semester_sep), semester_sep)
                    #_dbg_('111111', type(semester_sep[0]), len(semester_sep[0]), semester_sep[0])
                    #cnt += 1
                
                    #SQL Select로 검색된 분교 결과(rs_branch_dict_list)에 대해서 09xx 통합코드로 더하기 수행
                    tmp_val_dict = combi_3key_process(curs, conn, dbname, table_name, items, unames, combination_rule)
                    
                    
                    if not tmp_val_dict:
                        _dbg_("딕셔너리가 비어 있습니다.")
                    else:
                        _dbg_('\n신규 입력할 합산 결과 ', tmp_val_dict)
                        #통합된 신규 09xxx 레코드 인서트
                        insert_new_09xx(curs, conn, dbname, table_name, tmp_val_dict)
                        
            ret = conn.commit()
            _dbg_('SQL커밋 결과 ', ret)
        
        #sys.exit(0) 

#################################################
# 캠퍼스들을 통합 대학코드로 통합
#################################################
def run_combination_campus(curs, dbname, combination_rule):
    
    result_list2 = [] #2개 키
    result_list3 = [] #3개 키
    result_list_except = [] #3개 키
    
    #1. combination_rule
    for row in range(len(combination_rule)):
        for col in range(len(combination_rule[row])):
            print(combination_rule[row][col], end=" | ")
        print()
        
    #2. DB의 전체 테이블 리스트 읽어오기
    #table_dic_list = get_db_table_list(curs, dbname)
    
    #중도탈락만 대상으로 테스트
    table_dic_list = ['dropout_status']
    
    for n in range(len(table_dic_list)):
        #3. 각 테이블의 pk키 리스트 읽어오기 >> year 컬럼이 있는 테이블들만을 대상으로 통합작업을 수행해야함
        pk_list = get_db_table_keys(curs, dbname, table_dic_list[n])

        #4. pk_list중 year 키를 포함하는 테이블들만 추출
        # 키 2개 : field_training ['year', 'univ_cd'] [2021, 2022] | ...
        # 키 3개 : teaching_profession_grades ['year', 'univ_cd', 'semester'] [2020, 2021, 2022] | dormitory_result_private_univ_capital ['year', 'univ_cd', 'dormitory_type'] [2020, 2021, 2022] | ...
        # 키 4개 : instructor_fee ['year', 'univ_cd', 'instructor_classification', 'time classification', 'hour_price'] | dormitory_status ['year', 'univ_cd', 'dormitory_name', 'room_tot', 'applicant_other_area'] [2021, 2022, 2023] | ...
        if 'year' in pk_list:
            year_list = get_db_year_col(curs, dbname, table_dic_list[n])
            _dbg_(n, ' : ', table_dic_list[n], pk_list, year_list)
            
        #5. 현재까지 검색된 table_dic_list[n] 만 캠퍼스 통합 작업 대상 임
        #   combination_rule에 [year + univ_code] 기준으로만 통합할 캠퍼스들이 있는지 확인 >> 통합에는 단일 학교 코드로만 통합하는것이기 때문에...
        #   ex) 2023+0000001 과 2023+0000002가 존재한다면 각 행의 데이터들을 합해서 0900001에 삽입함
        #   이제 table_dic_list[n] 테이블에서 year + univ_code(combination_rule[2] 이상)가 존재하는지를 확인
        
        #5-1. find_combination_code
        
        
        if table_dic_list[n] not in except_table: # 예외테이블에 대해서는 작업을 진행하지 않음
            
            #테이블별로 모든 년도에 해당하는 모든 09XXXX 삭제
            #delete_old_09xx(curs, conn, dbname, table_name, year, combination_rule[unames][1])
            delete_old_09xx(curs, conn, dbname, table_dic_list[n], year_list, combination_rule)            
            
            key_cnt, key_list = find_key3_cnt(dbname, table_dic_list[n])
            if key_cnt == 3: #키가 3개인 경우에 대해서만 처리해보자
                if table_dic_list[n] in key3_table: #오류로 처리가 안된 에러테이블에 대한 임시작업용
                    univ_combination_3code(dbname, table_dic_list[n], year_list, combination_rule) #키 3개
                    result_list3.append(table_dic_list[n])
                
            elif key_cnt == 2: #키가 2개인 경우에 대해서만 처리해보자
                univ_combination_code(dbname, table_dic_list[n], year_list, combination_rule) #키 2개
                result_list2.append(table_dic_list[n])

            # elif table_dic_list[n] in key3_table: #키3개 테이블
            #     univ_combination_3code(dbname, table_dic_list[n], year_list, combination_rule)
            # else: #키2개 테이블
            #     univ_combination_code(dbname, table_dic_list[n], year_list, combination_rule)
        else:
            #_dbg_(table_dic_list[n], '테이블은 통합 작업을하지 않음')
            result_list_except.append(table_dic_list[n])
            # if table_dic_list[n] != 'group_info' and table_dic_list[n] != 'group_univ' and table_dic_list[n] != 'employment_type' and table_dic_list[n] != 'swu_fund' and table_dic_list[n] != 'university'and table_dic_list[n] != 'university_old' and table_dic_list[n] != 'swu_fund' and table_dic_list[n] != 'vw_education_expenses' and table_dic_list[n] != 'vw_professor_acquisition_rate_per_student_ranking' and table_dic_list[n] != 'vw_field_training':
            
        
        
    _dbg_('#########################################################################################################################################################')
    _dbg_('### 통합 작업 완료 ')
    _dbg_('### 키2개 테이블 수 : ', len(result_list2))
    _dbg_('### 키3개 테이블 수 : ', len(result_list3))
    _dbg_('### 미작업 테이블 수 : ', len(result_list_except))
    _dbg_('### 키2개 테이블 : ', result_list2)
    _dbg_('### 키3개 테이블 : ', result_list3)
    _dbg_('### 미작업 테이블 : ', result_list_except)
    _dbg_('#########################################################################################################################################################')
        
        #     case 1 : year + univ_code(combination_rule[2] 또는 year + univ_code(combination_rule[3] 또는 year + univ_code(combination_rule[4] 중 1개만 있다면 year + univ_code(combination_rule[1]의 코드에 단순 복사
        #     case 2 : year + univ_code(combination_rule[2] 또는 year + univ_code(combination_rule[3] 또는 year + univ_code(combination_rule[4] 중 2개 이상이라면 year + univ_code(combination_rule[1]의 코드에 각 컬럽의 합계를 삽입
        
run_combination_campus(curs, dbname, combination_rule)

In [ ]:
# 단독 작업
# 아래 오류와 같이 ...'0900002'와 같이 있어서는 안되는 코드가 들어가고 있다.
# [dbg] ( combination_campus )  0  :  field_training ['year', 'univ_cd'] [2021, 2022]
# [dbg] ( find_combination_code )  insert into swu_external.field_training(year,univ_cd, ft_4w_tot, ft_4w_insure_wound, ft_4w_insure_industrial, ft_4w_fee_75p, ft_4w_fee_100p, ft_4w_home_25p_under, ft_4w_home_25p_above, ft_8w_tot, ft_8w_insure_wound, ft_8w_insure_industrial, ft_8w_fee_75p, ft_8w_fee_100p, ft_8w_home_25p_under, ft_8w_home_25p_above, ft_12w_tot, ft_12w_insure_wound, ft_12w_insure_industrial, ft_12w_fee_75p, ft_12w_fee_100p, ft_12w_home_25p_under, ft_12w_home_25p_above) values(2021, '0900001', 4, 4, 4, 4, 0, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    
# [dbg] ( find_combination_code )  insert into swu_external.field_training(year,univ_cd, ft_4w_tot, ft_4w_insure_wound, ft_4w_insure_industrial, ft_4w_fee_75p, ft_4w_fee_100p, ft_4w_home_25p_under, ft_4w_home_25p_above, ft_8w_tot, ft_8w_insure_wound, ft_8w_insure_industrial, ft_8w_fee_75p, ft_8w_fee_100p, ft_8w_home_25p_under, ft_8w_home_25p_above, ft_12w_tot, ft_12w_insure_wound, ft_12w_insure_industrial, ft_12w_fee_75p, ft_12w_fee_100p, ft_12w_home_25p_under, ft_12w_home_25p_above) values(2022, '0900001', 37, 37, 37, 12, 25, 0, 0, 57, 57, 57, 14, 43, 0, 0, 15, 15, 15, 8, 7, 0, 0)    
# [dbg] ( find_combination_code )  insert into swu_external.field_training(year,univ_cd, ft_4w_tot, ft_4w_insure_wound, ft_4w_insure_industrial, ft_4w_fee_75p, ft_4w_fee_100p, ft_4w_home_25p_under, ft_4w_home_25p_above, ft_8w_tot, ft_8w_insure_wound, ft_8w_insure_industrial, ft_8w_fee_75p, ft_8w_fee_100p, ft_8w_home_25p_under, ft_8w_home_25p_above, ft_12w_tot, ft_12w_insure_wound, ft_12w_insure_industrial, ft_12w_fee_75p, ft_12w_fee_100p, ft_12w_home_25p_under, ft_12w_home_25p_above) values(2022, '0900002', 10, 10, 10, 2, 8, 0, 0, 5, 5, 5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0)




In [ ]:
#####################################################################################################################################################################################################################################################
#####################################################################################################################################################################################################################################################
# 학교 캠퍼스를 하나의 학교로 데이터를 통합하는 모듈 (기존데이터는 그대로 두고 신규 09XXXXXXX 인 코드를 만들어서 09번 코드학교에 합계를 입력) ====> 240715, 비율 부분만 별도 작업 수행
# 09 코드로 시작되는 부분만 읽어서 중도탈락수 * 재학생수 * 100으로 계산해서 dropout_case_rate와 fresh_dropout_case_rate에 업데이트를 해야함
# 1. dropout_case_tot / enrolled_tot * 100 을 dropout_case_rate
# 2. fresh_dropout_case_tot / fresh_enrolled_tot * 100 을 fresh_dropout_case_rate
#####################################################################################################################################################################################################################################################
#####################################################################################################################################################################################################################################################

In [ ]:
os.exit(0)

In [ ]:
table_dic_list = get_db_table_list(curs, dbname)

# for n in range(len(table_dic_list)):
#     print(table_dic_list[n]['table_name'])
    

# DB 테이블의 컬럼명과 데이터타입만 추출
col_info, col_name_str, col_name_list = get_db_column_name(curs, dbname, 'enrolled_student')

In [ ]:
pk_list = get_db_table_keys(curs, dbname, 'enrolled_student')

print(pk_list)
# for n in range(len(key_dic_list)):
#     print(key_dic_list[n]['table_name'])

In [ ]:
chk_table_and_rule(curs, dbname, 'enrolled_student', combination_rule)

In [ ]:
get_db_year_col(curs, dbname, 'enrolled_student')

In [ ]:
#################################################
# DB 테이블의 컬럼명과 데이터타입만 추출
#################################################
def get_db_column_name(curs, dbname, table):
    _dbg_(table)
    #cnt_columns = len(df_excel.columns) #엑셀 df읠 필드명

    sql_col = 'SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = "{}"'.format(table)

    curs.execute(sql_col)
    #conn.commit()
    col_info = curs.fetchall() #컬럼명과 데이터타입 리스트 안에 딕셔너리로...
    
    _dbg_(table, '테이블의 컬럼 정보 : ', type(col_info), len(col_info), col_info[:8])

    idx_mode = 'off' #기준연도+학교코드 모드와 idx 모드로 나뉨
    col_name_list = list()
    col_name_str = ''
    col_cnt = len(col_info)
    
    for n in range(col_cnt):
        col = col_info[n]['COLUMN_NAME']
        #_dbg_('[{}]'.format(col))
        if col == 'idx':
            idx_mode = 'on'
        else:
            #col_name_list.append(col_name[n]['COLUMN_NAME'])
            col_name_list.append(col)
            #_dbg_('appended : [{}]'.format(col))

    if idx_mode == 'on': #idx하나를 빼야 함
        col_cnt = col_cnt - 1

    for n in range(col_cnt):
        if n == 0:
            col_name_str = '`' + str(col_name_list[n]) + '`, '
        elif n >= col_cnt-1:
            col_name_str = col_name_str + '`' + str(col_name_list[n]) + '`'
        else:
            col_name_str = col_name_str + '`' + str(col_name_list[n]) + '`, '


    #print(col_name_str)
    # print(type(col_name), len(col_name), col_name)
    # print(type(col_name[0]), len(col_name[0]), col_name[0])
    # print(type(col_name[0]['COLUMN_NAME']), col_name[0]['COLUMN_NAME'])

    return col_info, col_name_str, col_name_list

In [ ]:
'''

#####################################################################################################################################################################################################################################################
# 모든 테이블의 기존 임시부여한 키값을 새로운 키 값으로 업데이트
#####################################################################################################################################################################################################################################################


#################################################
# 테이블의 univ_cd key 에 해당하는 전체 컬럼값을 읽은 후, old 코드라면 새로운 코드로 변경
#################################################
def update_db_univ_cd_col(curs, dbname, table_name):
    #key_list = get_db_table_keys(curs, dbname, table_name)
    #sql = "SELECT {} FROM {}.{}".format('univ_cd', dbname, table_name)
    
    #_dbg_(old_and_new_univ_cd[0][1])
    
    update_cnt = 0
    for n in range(len(old_and_new_univ_cd)):
        sql = "UPDATE {}.{} SET univ_cd = '{}' WHERE univ_cd = '{}'".format(dbname, table_name, old_and_new_univ_cd[n][1], old_and_new_univ_cd[n][0])
        
        rs_cnt = curs.execute(sql)
        #rs_dic_list = curs.fetchall() #결과 목록
        
        
        update_cnt = update_cnt + rs_cnt
        if rs_cnt >= 1:
            _dbg_(sql)    
            conn.commit()
    
    if update_cnt > 1:
        _dbg_(update_cnt, '개의 업데이트 성공 #######################')


#1. DB의 전체 테이블 리스트 읽어오기
table_dic_list = get_db_table_list(curs, dbname)

_dbg_(table_dic_list)

for n in range(len(table_dic_list)):
    #2. 각 테이블에서 univ_cd 컬럼의 값들만 읽어옴
    if table_dic_list[n] != 'group_info' and table_dic_list[n] != 'group_univ' and table_dic_list[n] != 'employment_type' and table_dic_list[n] != 'swu_fund' and table_dic_list[n] != 'university'and table_dic_list[n] != 'university_old' and table_dic_list[n] != 'swu_fund' and table_dic_list[n] != 'vw_education_expenses' and table_dic_list[n] != 'vw_professor_acquisition_rate_per_student_ranking' and table_dic_list[n] != 'vw_field_training':
    #         if table_dic_list[n] != 'graduate_employment_status': # and table_dic_list[n] != 'enrolled_student' and table_dic_list[n] != 'graduate_grades' and table_dic_list[n] != 'foreign_professor_department_status' and table_dic_list[n] != 'professor_acquisition_rate_per_student' and table_dic_list[n] != 'employee_status' and  table_dic_list[n] != 'student_recruitment_rate' and table_dic_list[n] != 'field_training':
                
    #if table_dic_list[n] == 'professor_acquisition_rate_per_student':# or table_dic_list[n] == '' or table_dic_list[n] == '' or table_dic_list[n] != '' or table_dic_list[n] != '' or table_dic_list[n] != 'employee_status':
        _dbg_(table_dic_list[n], '테이블 작업')
        update_db_univ_cd_col(curs, dbname, table_dic_list[n])

'''